In [1]:
import random
import math
class Cell:
  def __init__(self, x, y, value, id):
    self.alien_id = id
    self.x = x
    self.y = y
    self.value = value
    self.parent = None
    self.left = None
    self.right = None
    self.top = None
    self.bottom = None
    self.visited = False
    self.neighbors = []
    self.open = False
    self.loc = [x, y]
    self.alien = False
    self.alien_probability = 0
    self.cell_in_detector = False
    self.crewmate_probability = .5
    self.prob_alien_moved_from = 0
    self.prob_alien_moved_to = 0
    self.move_value = 0

def initialize_cell_neighbors(grid):
  rows=len(grid)
  cols=len(grid)
  size=len(grid)

  for i in range(rows):
    for j in range(cols):
      # print("CURRENT",[i,j])
      left=j-1
      right=j+1
      top=i-1
      bottom=i+1
      #Testing for valid neighbor coordinates
      if (top<0):
        # print("CHECK T")
        top=-1
      if (bottom==size):
        # print("CHECK B")
        bottom=-1
      if (left<0):
        # print("CHECK L")
        left=-1
      if (right==size):
        # print("CHECK R")
        right=-1

      # print([left,right,top,bottom])

      if (left!=-1):
        grid[i][j].left=grid[i][left]
      else:
        grid[i][j].left = None
      if (right!=-1):
        grid[i][j].right=grid[i][right]
      else:
        grid[i][j].right = None
      if (top!=-1):
        grid[i][j].top=grid[top][j]
      else:
        grid[i][j].top = None
      if (bottom!=-1):
        grid[i][j].bottom=grid[bottom][j]
      else:
        grid[i][j].bottom = None

  return grid

def create_grid(rows, cols):
    grid = [[Cell(i, j, 0, 3) for j in range(cols)] for i in range(rows)]
    return grid

def is_valid(cell, grid):
  if(0 <= cell.x < len(grid)):
    if(0 <= cell.y < len(grid[0])):
      return True
  return False

def get_neighbors(cell, grid):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
    neighbors = []

    for dx, dy in directions:
        new_x, new_y = cell.x + dx, cell.y + dy
        if is_valid(Cell(new_x, new_y, '', 3), grid):
          #if (grid[new_x][new_y].open):
            neighbors.append(grid[new_x][new_y])

    return neighbors

def get_open_neighbors(cell, grid):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
    open_neighbors = []

    for dx, dy in directions:
        new_x, new_y = cell.x + dx, cell.y + dy
        if is_valid(Cell(new_x, new_y, '', 3), grid):
          if (grid[new_x][new_y].open):
            open_neighbors.append(grid[new_x][new_y])

    return open_neighbors

def detect_if_cell_open(cell):
  if (cell.value == 1):
    return True
  if (cell.value == 'A'):
    return True
  if (cell.value == 'C'):
    return True
  if (cell.value == 'B'):
    return True
  if (cell.value == 0):
    return False

def detect_open_cells(grid):
  open_cells = []
  for row in grid:
    for cell in row:
      if (cell.open):
        open_cells.append(cell)
  return open_cells

def initialize_detected_cells(grid, bot_cell, detector_size):
  count=0
  for row in grid:
    for cell in row:
      if (bot_cell.x - detector_size) <= cell.x <= (bot_cell.x + detector_size) :
        if (bot_cell.y - detector_size) <= cell.y <= (bot_cell.y + detector_size):
          cell.cell_in_detector = True
  return grid

def initialize_aliens(num_aliens, grid, bot_cell, detector_size):
  aliens = []                                                # initialize initialize_ship function                                                            # initialize matrix
  current_alien_start_cells = []
  open_cells = []
  grid=initialize_detected_cells(grid, bot_cell, detector_size)
  for row in grid:
    for cell in row:
      if (cell.open):
        if not cell.cell_in_detector:
          # print("OPEN CELL OUTSIDE DET GRID",[cell.x,cell.y])
          open_cells.append(cell)

  for i in range(num_aliens):
    alien_start_cell = random.choice(open_cells)
    while (alien_start_cell in current_alien_start_cells):
      alien_start_cell = random.choice(open_cells)
    current_alien_start_cells.append(alien_start_cell)
    alien_start_cell.id = i
    alien_start_cell.alien = True
    alien_start_cell.value = 'A'
    aliens.append(alien_start_cell)

  for row in grid:
    for cell in row:
      if (cell.open):
        if not cell.cell_in_detector:
          cell.alien_probability = 1/len(open_cells)
        else:
          cell.alien_probability=0
  return aliens

def initialize_prob_crewmate(grid, num_crewmate):
  num_open_cells = len(detect_open_cells(grid))
  for row in grid:
    for cell in row:
      if cell.open and cell.value != 'B':
        cell.crewmate_probability = 1/(num_open_cells-1)
      elif cell.open and cell.value == 'B':
        cell.crewmate_probability = 0
      else:
        cell.crewmate_probability = 0

def initialize_prob_alien_moved(grid):
  for row in grid:
    for cell in row:
      if cell.open:
        if cell.neighbors:
          cell.prob_alien_moved_from = 1/len(cell.neighbors)
        else:
          cell.prob_alien_moved_from = 0



In [2]:
import heapq
def dijkstra(grid,source):
  distances = {}
  predecessors={}
  for i in range(len(grid)):
    for j in range(len(grid[0])):
        distances[(i,j)]=float('inf')
        predecessors[(i,j)]=None
  distances[source] = 0
  priority_queue=[[0,source]]
  while priority_queue:
    current_distance, current_vertex = heapq.heappop(priority_queue)
        # Check if the current path is shorter than the known shortest path
    if current_distance > distances[current_vertex]:
        continue

    for neighbor in get_open_neighbors(grid[current_vertex[0]][current_vertex[1]],grid):
        neighbor=(neighbor.loc[0],neighbor.loc[1])
        distance = current_distance + 1

              # If a shorter path is found, update distances and predecessors
        if distance < distances[neighbor]:
          distances[neighbor] = distance
          predecessors[neighbor] = current_vertex
          heapq.heappush(priority_queue, (distance, neighbor))

  return distances, predecessors

In [3]:
import pandas as pd

def draw_ship_df(grid):


  columns_123=[]
  rows_123=[]
  for i in range(len(grid)):
    columns_123.append(i)
    rows_123.append(i)

  values_matrix=[]

  for row in grid:
    temp_row=[]
    for cell in row:
      edit_val=""
      if (cell.value==1 and cell.crewmate_probability != 0):
        edit_val="o"
      elif (cell.value==1 and cell.crewmate_probability == 0):
        edit_val = "O"
      elif (cell.value == 'C'):
        edit_val="C"
      elif (cell.value == 'A'):
        edit_val="A"
      elif (cell.value == 'C*'):
        edit_val="c"
      elif (cell.value == 'B'):
        edit_val="B"
      elif (cell.value == 'D'):
        edit_val="D"
      else:
        edit_val="x"
      temp_row.append(edit_val)
    values_matrix.append(temp_row)

  df = pd.DataFrame(values_matrix, columns=columns_123, index=rows_123)
  print(df.to_string())
  print("\n")

In [4]:
def detect_possible_future_cells(open_cell, possible_future_cells, n):
  for neighbor in open_cell.neighbors:

    counter=0

    open_neighbor_count=0

    for neighbor_neighbor in neighbor.neighbors:

      if (neighbor_neighbor.value==1):
        counter=counter+1

      if (neighbor_neighbor.value==1):
        open_neighbor_count=open_neighbor_count+1

    if (counter<2 and open_neighbor_count>=1):
      possible_future_cells.append(neighbor)
  return possible_future_cells

In [5]:
def back_check_possible_future_cells(possible_future_cells):
  for cell in possible_future_cells:
    counter=0
    for neighbor in cell.neighbors:
      if (neighbor.value==1):
        counter=counter+1
    if (counter>1):
      possible_future_cells.remove(cell)
  return possible_future_cells

In [6]:
def find_dead_ends(grid, n):
  dead_ends=[]
  for row in grid:
    for cell in row:
      if cell.value==1:
        open_count=0
        for neighbor in cell.neighbors:
          if neighbor.value==1:
            open_count=open_count+1
        if (open_count==1):
          dead_ends.append(cell)
  return dead_ends

In [7]:
def open_half_of_dead_ends(grid, dead_ends, n):
  num_dead_ends=len(dead_ends)
  nodes_to_open=int(num_dead_ends/2+1)
  for i in range(nodes_to_open):
    try:
      dead_end_to_open = random.choice(dead_ends)
    except:
      continue
    #arr=['left','right','top','bottom']
    valid_neighbors_to_open=[]
    for neighbor in dead_end_to_open.neighbors:
      #if (dead_end_to_open[i][0]==-1 or dead_end_to_open[i][1]==-1 or dead_end_to_open[i][0]==n or dead_end_to_open[i][1]==n):
        #continue
      if (neighbor.value==1):
        continue
      # elif (neighbor['self'][0]==-1 or neighbor['self'][1]==-1 or neighbor['self'][0]==n or neighbor['self'][1]==n):
        # continue
      else:
        valid_neighbors_to_open.append(neighbor)
    if len(valid_neighbors_to_open)!=0:
      dead_end_neighbor_to_open = random.choice(valid_neighbors_to_open)
    dead_end_neighbor_to_open.value=1
    dead_end_neighbor_to_open.open=True
    dead_ends.remove(dead_end_to_open)
  return grid, dead_ends

In [8]:
import random
def explore_grid(n, grid, num_aliens, num_crew, detector_size):                                                    # initialize function
  # Generate two random values within the range [1, n]
  random_value1 = random.randint(1, n-2)                                        # generate random y value
  random_value2 = random.randint(1, n-2)                                        # generate random x value

  starting_cell=grid[random_value1][random_value2]                            # initialize open_cell
  # Open First Cell
  starting_cell.value = 1         # Open First Cell
  current_open_cell=starting_cell                                               # initialize current_open_cell as open_cell
  possible_future_cells=[]                                                      # initialize possible_future_cells

  while True:
    if possible_future_cells:                                                 # if possible_future_cells not empty
      new_chosen_cell = random.choice(possible_future_cells)                  # initialize new_chosen_cell as random cell from possible_future_cells
      new_chosen_cell.value = 1                                               #open the new_choesn_cell
      new_chosen_cell.open = True                                                                                                                                   #Temporary fix should be fixed in class def i think
      current_open_cell=new_chosen_cell                                       # set current_open_cell to be the newest open cell
      possible_future_cells.remove(new_chosen_cell)                           # remove new_chosen_cell from possible_future_cells
    possible_future_cells = detect_possible_future_cells(current_open_cell, possible_future_cells, n) # add valid neighbors of current_open_cell to possible_future_cells
    if (starting_cell in possible_future_cells):
        possible_future_cells.remove(starting_cell)
    possible_future_cells = back_check_possible_future_cells(possible_future_cells) # remove invalid cells from possible_future_cells
    if (starting_cell in possible_future_cells):
        possible_future_cells.remove(starting_cell)

    if (len(possible_future_cells)==0):                                       # if no more possible_future_cells to add end while loop
      break

  dead_ends=find_dead_ends(grid, n)
  grid, des = open_half_of_dead_ends(grid, dead_ends, n)

  bot_cell = create_bot(grid)
  aliens = initialize_aliens(num_aliens, grid, bot_cell, detector_size)

  #draw_ship_df(grid)
  #draw_alien_probabilities_matrix(grid)


  for x in range(num_crew):
    possible_crewmate_cells = detect_open_cells(grid)
    possible_crewmate_cells.remove(bot_cell)
    crewmate_cell = random.choice(possible_crewmate_cells)
    crewmates.append(crewmate_cell)
    crewmate_cell.value = 'C'

  # if (len(detect_open_neighbors_bot_1(matrix[crew_start_cell[0]][crew_start_cell[1]]))==0):
    # modify_blocked_cells(matrix,crew_start_cell)

  return aliens, crewmates, bot_cell                                                               # return completed matrix

In [9]:
def create_bot(grid):
  open_cells=detect_open_cells(grid)
  bot_start_cell=random.choice(open_cells)
  bot_start_cell.value = 'B'
  bot_start_cell.crewmate_probability = 0
  return bot_start_cell

In [10]:
def take_step(grid, aliens, crewmates, bot_cell,time_steps):
  # print("From start of take_step: Bot position:", bot_cell, "Bot value", matrix[bot_cell[0]][bot_cell[1]]['value'])                           # VERY HELPFUL FOR DEBUG
  # print("From take_step: Crew position", crew_start_cell)                                                                                     # VERY HELPFUL FOR DEBUG
  import random
  import copy
  #print("take step called")
  time_steps+=1

  if bot_id == 1 or bot_id == 2 or bot_id == 3 or bot_id == 4 or bot_id == 5 or bot_id == 6:
    alien_beep = alien_detector(grid, bot_cell, detector_size)
    #print("alien detected within take step: ", alien_beep)

    detect_alien_probabilities(grid)

    #print("Alien Prob Matrix:")
    #draw_alien_probabilities_matrix(grid)

  if bot_id == 7 or bot_id == 8:
    alien_beep = alien_detector(grid, bot_cell, detector_size)
    #print("alien detected within take step: ", alien_beep)

    detect_alien_pairs_probs(grid, alien_beep)

    #print("Alien Prob Matrix:")
    #draw_alien_probabilities_matrix(grid)


  new_aliens=[]

  random.shuffle(aliens)
  for current_alien in aliens:
    # old_alien_location = [alien.x, alien.y]                                                                                                    # VERY HELPFUL FOR DEBUG
    # print("From take_step: Alien", i["alien_id"], "action: ")                                                                                 # VERY HELPFUL FOR DEBUG
    for crew_cell in crewmates:
      # if (crew_cell.value == 'B'):  # Commenting this out since project 2 doesnt have to reinitialize crewmates
        #print("From take_step: Bot found crew memeber loading new crew memeber")                                                                # VERY HELPFUL FOR DEBUG
        # crew_cell = random.choice(detect_open_cells(grid)) # Commenting this out since project 2 doesnt have to reinitialize crewmates

        # if (len(detect_open_neighbors_bot_1(matrix[crew_start_cell[0]][crew_start_cell[1]]))==0):
          # modify_blocked_cells(matrix,crew_start_cell)
        #print("Frome take_step: New Crew position: ", crew_cell)                                                                          # VERY HELPFUL FOR DEBUG
        # grid[crew_cell.x][crew_cell.y].value = 'C' # Commenting this out since project 2 doesnt have to reinitialize crewmates
        #crew_cell.value = 'C'

      for other_alien in aliens:
        if current_alien.id != other_alien.id:
          if (current_alien.loc == other_alien.loc):
            grid[current_alien.x][current_alien.y].value = 'A'
      if (current_alien.loc == bot_cell.loc):                #Bot is moving into an alien cell at the same time as the alien moves? not sure if this is correct
        grid[current_alien.x][current_alien.y].value = 'B'                              #THIS DEFFINITLY IS WRONG
        #current_alien.value = 'B'

      if (current_alien.loc == crew_cell.loc):
        grid[current_alien.x][current_alien.y].value = 'C'
        #aliens[i].value = 'C'
      else:
        #aliens[i].value = 1
        grid[current_alien.x][current_alien.y].value = 1

    #print("From take_step: Current Alien location: ", aliens[i].loc)                                                                             # VERY HELPFUL FOR DEBUG

    current_alien.open = True
    current_alien.id = 3
    current_alien.alien = False
    grid[current_alien.x][current_alien.y].open = True
    # print("CHECK1 for current alien",grid[current_alien.x][current_alien.y].alien)
    grid[current_alien.x][current_alien.y].alien = False
    x_old=current_alien.x
    y_old=current_alien.y
    current_alien.neighbors = get_open_neighbors(current_alien, grid)
    # print("CHECK2 for current alien",grid[current_alien.x][current_alien.y].alien)

    # print("CHECK1 for current alien loc",current_alien.loc)

    try:
      alien_move = random.choice(current_alien.neighbors)
    except:
      for alien in aliens:
        alien.neighbors = get_open_neighbors(alien, grid)
      alien_move = random.choice(current_alien.neighbors)

    # print("BIG CHECK1",vars(current_alien))
    current_alien=grid[alien_move.x][alien_move.y]
    # print("BIG CHECK2",vars(current_alien))
    current_alien.neighbors = get_open_neighbors(alien_move, grid)
    current_alien.x = alien_move.x
    current_alien.y = alien_move.y
    current_alien.loc = [alien_move.x, alien_move.y]
    # current_alien.value = 'A'

    # current_alien.open = False

    # # Updating the values for current alien to new choice
    # current_alien.id = 0
    # current_alien.alien = True

    # # Updating values of old cell
    # grid[x_old][y_old].alien = False
    # grid[x_old][y_old].id = 0

    x1=x_old
    y1=y_old


    # print("CHECK3 for current alien",grid[x1][y1].alien)
    # print("C x1",current_alien.x,"C y1",current_alien.y)
    grid[current_alien.x][current_alien.y].alien = True
    # Updating the values for current alien to new choice
    current_alien.id = 0
    current_alien.alien = True

        # Updating values of old cell
    grid[x_old][y_old].alien = False
    grid[x_old][y_old].id = 0

    grid[current_alien.x][current_alien.y].alien = True
    grid[current_alien.x][current_alien.y].id = 0

    # print("CHECK4 for current alien",grid[x1][y1].alien)

    # print("CHECK2 for current alien loc",current_alien.loc)


    #print("From take_step: New Alien location: ", aliens[i].loc)                                                                                 # VERY HELPFUL FOR DEBUG

      #current_alien.value = 'A'
      # print("From take_step: except:")                                                                                                        # VERY HELPFUL FOR DEBUG
      # print("From take_step: Ship after alien", i["alien_id"], "moves")                                                                       # VERY HELPFUL FOR DEBUG
      # draw_ship(matrix)

    for crew_cell in crewmates:                                                                                                                      # VERY HELPFUL FOR DEBUG
      if current_alien == crew_cell:
        #aliens[i].value = 'C*'
        grid[current_alien.x][current_alien.y].value = 'C*'
        current_alien.value = 'C*'

    else:
      #aliens[i].value = 'A'
      grid[current_alien.x][current_alien.y].value = 'A'
      current_alien.value = 'A'

    # print("From take_step: Ship after alien", i["alien_id"], "move")                                                                          # VERY HELPFUL FOR DEBUG
    # print("From take_step: ", i["alien_id"], "moves from", old_alien_location, "to ", i['self'])                                              # VERY HELPFUL FOR DEBUG
    # print("From take_step: New cell value", matrix[i['self'][0]][i['self'][1]]['value'])                                                      # VERY HELPFUL FOR DEBUG
    # draw_ship(matrix)                                                                                                                         # VERY HELPFUL FOR DEBUG
    # print("----------------------")                                                                                                           # VERY HELPFUL FOR DEBUG
  # print("From end of take_step: Bot position:", bot_cell, "Bot value", matrix[bot_cell[0]][bot_cell[1]]['value'])                             # VERY HELPFUL FOR DEBUG
    for alien_cell in aliens:
      if bot_cell == alien_cell:                                         # VERY HELPFUL FOR DEBUG
        #print("From take_step: take_step has detected the bot being caught by an alien: Bot location: ", bot_cell, "Alien cell: ", aliens[i].loc)             # VERY HELPFUL FOR DEBUG
        #draw_ship_df(grid)
        new_aliens.append(current_alien)
        return "The bot was caught by the aliens!", crewmates, new_aliens,time_steps, alien_beep

    # print("BIG CHECK3",vars(current_alien))
    new_aliens.append(current_alien)


  aliens=new_aliens
  # print("BIG CHECK 4",vars(aliens[0]))
  #print("Ship after time step:", time_steps)
  #draw_ship_df(grid)

  if bot_id == 1 or bot_id == 2 or bot_id == 3  or bot_id == 4 or bot_id == 5 or bot_id == 6:
    alien_move_prob(grid)
    for row in grid:
      for cell in row:
        cell.alien_probability = cell.prob_alien_moved_to

    #print("Alien Prob Matrix after prob moved:")
    #draw_alien_probabilities_matrix(grid)

  if bot_id == 7 or bot_id == 8:
    alien_pairs_move_probs(grid)

    #print("Alien Prob Matrix after prob moved:")
    #draw_alien_probabilities_matrix(grid)

  if bot_id == 1 or bot_id == 2 or bot_id == 3  or bot_id == 6:
    crewmate_beep = detect_crewmate_beep(grid, crewmates, bot_cell, alpha,time_steps)
    #print("Crewmate_beep = ", crewmate_beep)
    detect_crewmate_prob(grid, bot_cell, crewmate_beep, time_steps, crewmates)

  if bot_id == 4 or bot_id == 5 or bot_id == 7 or bot_id == 8:
    crewmate_beep = detect_crewmate_beep(grid, crewmates, bot_cell, alpha,time_steps)
    #print("Crewmate_beep = ", crewmate_beep)
    detect_crewmate_pairs_probs(grid, bot_cell, crewmate_beep, time_steps)

  # print("CREW PROBS MATRIX")
  # draw_crewmate_probabilities_matrix(grid)
  # print("Sum of Alien Probabilities")
  # print(sum_probabilities_alien(grid))
  # print("Sum of Crewmate Probabilities")
  # print(sum_probabilities_crewmate(grid))
  # print("----------------------------------------------------------------------------------")                                                 # VERY HELPFUL FOR DEBUG

  return "Status normal", crewmates, new_aliens,time_steps, alien_beep

In [11]:
def detect_if_crew_memeber_blocked(grid, crew_start_cell) :
  if (crew_start_cell.neighbors):
    print("Not Blocked")
  else:
    print("Blocked")

In [12]:
def alien_detector(grid, bot_cell, detector_size) :
  #print("Alien detector called")
  alien_beep = False
  for row in grid:
    for cell in row:
      cell.cell_in_detector = False
      if (bot_cell.x - detector_size) <= cell.x <= (bot_cell.x + detector_size) :
        if (bot_cell.y - detector_size) <= cell.y <= (bot_cell.y + detector_size):
          #print("CELL IN DETECTOR",[cell.x,cell.y])
          cell.cell_in_detector = True
          #print("Cell in detector: ", cell.loc)
          # cell.value = 'D'

          if cell.alien == True:
            #print("ALIEN DETECTED")
            alien_beep = True
            #print("alien_beep set to ", alien_beep)
  #print("At end of alien_detector alien_beep = ", alien_beep)
  # print("Showing Detection Area")
  # draw_ship_df(grid)
  # for row in grid:
  #   for cell in row:
  #     if cell.value == 'D':
  #       cell.value = 1
  return alien_beep

In [13]:
def detect_alien_probabilities(grid):
  if alien_beep:
    #print("BEEP DETECTED")
    prob_alien_detected = 1
    cell_probs = []
    for row in grid:
      for cell in row:
        if cell.open:
          if not cell.cell_in_detector:
            #print("OPEN CELL NOT INSIDE DETECTOR",[cell.x,cell.y],"SETTING CELL PROB 0")
            cell.alien_probability = 0
    for row in grid:
      for cell in row:
        if cell.open:
          if cell.cell_in_detector:
            #print("PROB ASSIGNED TO OPEN CELL",[cell.x, cell.y], "IS",cell.alien_probability*prob_alien_detected)
            if cell.alien_probability==0:
              cell.alien_probability=0.00000001
            cell_probs.append(cell.alien_probability*prob_alien_detected)

    sum_of_probs = sum(cell_probs)
    #print("SUM OF PROBS IS",sum_of_probs)
    for row in grid:
      for cell in row:
        if cell.open:
          if cell.cell_in_detector:
            cell.alien_probability = (cell.alien_probability*prob_alien_detected)/(sum_of_probs*prob_alien_detected)
  else:
    #print("NO BEEP DETECTED")
    prob_alien_detected = 1
    cell_probs = []
    for row in grid:
      for cell in row:
        if cell.open:
          if cell.cell_in_detector:
            #print("OPEN CELL INSIDE DETECTOR",[cell.x,cell.y],"SETTING CELL PROB 0")
            cell.alien_probability = 0
    for row in grid:
      for cell in row:
        if cell.open:
          #print("PROB ASSIGNED TO OPEN CELL",[cell.x, cell.y], "IS",cell.alien_probability*prob_alien_detected)
          if cell.alien_probability==0:
              cell.alien_probability=0.00000001
          cell_probs.append(cell.alien_probability*prob_alien_detected)
    sum_of_probs = sum(cell_probs)
    #print("DENOM IS",sum_of_probs)
    for row in grid:
      for cell in row:
        if cell.open:
          if not cell.cell_in_detector:
            cell.alien_probability = (cell.alien_probability*prob_alien_detected)/(sum_of_probs*prob_alien_detected)
            #print("CELL ALIEN PROBABILITY FOR OPEN CELL NOT INSIDE DETECTOR",[cell.x,cell.y],"IS",cell.alien_probability)


In [14]:
def alien_move_prob(grid):
  for row in grid:
    for cell in row:
      if cell.open:
        denom_values = []
        x_prime_values = []
        for row in grid:
          for current_cell in row:
            if current_cell.open:
              for neighbor in current_cell.neighbors:
                # Neighbor.alien_probability is the prob of how likely the alien was present in that neighbor
                # neighbor.prob_alien_moved_from is how likely the current cell is the alien location given that this neighbor was the prev alien location
                denom_values.append(neighbor.alien_probability*neighbor.prob_alien_moved_from) #*probability_of_collecting_final_observation
        beta_t = 1/sum(denom_values)
        for neighbor in cell.neighbors:
            x_prime_values.append(neighbor.alien_probability*neighbor.prob_alien_moved_from) #*probability_of_collecting_final_observation
        x_prime = sum(x_prime_values)

        cell.prob_alien_moved_to = beta_t*x_prime
      else:
        cell.prob_alien_moved_to = 0

In [15]:
import numpy as np
def detect_crewmate_beep(grid, crewmates, bot_cell, alpha,time_steps):
  for crew_cell in crewmates:
    d=distance_map[(bot_cell.x,bot_cell.y)][(crew_cell.x,crew_cell.y)]
    p_crew = math.exp((-1)*alpha*(d-1)) #e−α(d−1)
    if p_crew>1:
      p_crew=0
    crewmate_beep_value = np.random.binomial(1, p_crew, 1)
    if crewmate_beep_value[0] == 1:
      return True
  else:
    return False

In [16]:
def detect_crewmate_prob(grid,bot_cell, crewmate_beep, time_steps, crewmates):
  if crewmate_beep:
    prob_crewmate_detected = 1
    cell_probs = []
    for row in grid:
      for cell in row:
        if cell.open:
          d=distance_map[(bot_cell.x,bot_cell.y)][(cell.x,cell.y)]
          p_beep = math.exp((-1)*alpha*(d)) #e−α(d−1)
          cell.crewmate_probability = p_beep*cell.crewmate_probability
          cell_probs.append(cell.crewmate_probability)
    sum_of_probs = sum(cell_probs)
    for row in grid:
      for cell in row:
        if cell.open:
          cell.crewmate_probability = (cell.crewmate_probability)/(sum_of_probs)
  if not crewmate_beep:
    prob_crewmate_detected = 1
    cell_probs = []
    for row in grid:
      for cell in row:
        if cell.open:
          d=distance_map[(bot_cell.x,bot_cell.y)][(cell.x,cell.y)]
          p_beep = (1 - math.exp((-1)*alpha*(d))) #e−α(d−1)
          cell.crewmate_probability = p_beep*cell.crewmate_probability
          cell_probs.append(cell.crewmate_probability)
    sum_of_probs = sum(cell_probs)
    for row in grid:
      for cell in row:
        if cell.open:
          cell.crewmate_probability = (cell.crewmate_probability)/(sum_of_probs)


In [17]:
from decimal import Decimal, getcontext
getcontext().prec = 20
# Function to add a key-value pair to the hashmap
# The key is a pair of coordinate pairs ((x, y), (a, b))
def add_coordinates_crewmate_pairs(x, y, a, b, value):
    crewmate_pairs_probs[((x, y), (a, b))] = value

# Function to get a value from the hashmap given a key
def get_value_crewmate_pairs(x, y, a, b):
    return crewmate_pairs_probs.get(((x, y), (a, b)), Decimal(1/((num_open_cells-1)*(num_open_cells-1))))


In [18]:
from decimal import Decimal, getcontext
getcontext().prec = 20
def initialize_crewmate_pairs_probs(crewmate_pairs_probs):
  num_open_cells = len(detect_open_cells(grid))
  prob = Decimal(1/((num_open_cells-2)*(num_open_cells-2)))
  for row in grid:
    for cellj in row:
      for row in grid:
        for cellk in row:
          if cellk != cellj:
            add_coordinates_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y, prob)

In [19]:
from decimal import Decimal, getcontext
getcontext().prec = 20
def detect_crewmate_pairs_probs(grid, bot_cell, crewmate_beep, time_steps):
  #Finding the sum of probablities i.e. normalization factor
  if crewmate_beep:
    denom_values = []
    #prob_beep_values = []
    for rowj in grid:
      for cellj in rowj:
        if cellj.open:
          if cellj != bot_cell:
            dj = distance_map[(bot_cell.x,bot_cell.y)][(cellj.x, cellj.y)]
            prob_beep_from_cellj = Decimal(math.exp((-1)*alpha*(dj-1)))
            for rowk in grid:
              for cellk in rowk:
                if cellk.open:
                  if cellk != cellj:
                    if cellk != bot_cell:
                      dk = distance_map[(bot_cell.x,bot_cell.y)][(cellk.x, cellk.y)]
                      prob_beep_from_cellk = Decimal(math.exp((-1)*alpha*(dk-1)))
                      prob_beep_from_cellj_and_cellk = Decimal(1-(1-prob_beep_from_cellj)*(1-prob_beep_from_cellk))
                      # if prob_beep_from_cellj_and_cellk < 0:
                      #   print("prob_beep less than 0")
                      #   print(prob_beep_from_cellj_and_cellk)
                      #   print("prob_bep_cellj: ", prob_beep_from_cellj)
                      #   print("dj: ", dj)
                      #   print("prob_bep_cellk: ", prob_beep_from_cellk)
                      #   print("dk: ", dk)
                      #   print("cellj: ", cellj.loc)
                      #   print("cellk: ", cellk.loc)
                      #   print("bot_cell: ", bot_cell.loc)
                      #   draw_ship_df(grid)
                      #   draw_crewmate_probabilities_matrix(grid)
                      # for key,val in crewmate_pairs_probs.items():      #FIX
                      #   if crewmate_pairs_probs[key] == 0:
                      #     crewmate_pairs_probs[key]=0.0000001
                      denom_values.append(Decimal(get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk))
                      # if get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk <= 0:
                      #   print("get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk: ", get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk)
                      #   print("get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y): ", get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
                      #   print("denom_value less than 0")
                      #   print("prob_beep_from_cellj_and_cellk: ", prob_beep_from_cellj_and_cellk)
                      #   print("prob_bep_cellj: ", prob_beep_from_cellj)
                      #   print("dj: ", dj)
                      #   print("prob_bep_cellk: ", prob_beep_from_cellk)
                      #   print("dk: ", dk)
                      #   print("cellj: ", cellj.loc)
                      #   print("cellk: ", cellk.loc)
                      #   print("bot_cell: ", bot_cell.loc)
                      #   draw_ship_df(grid)
                      #   draw_crewmate_probabilities_matrix(grid)
                      #   print("--------------------------------------------------")
                      # prob_beep_values.append(prob_beep_from_cellj_and_cellk)
    sum_denom = sum(denom_values)
    # if sum_denom == 0:
    #   print("###################################")
    #   print("Divide by Zero Error:")
    #   print("Sum_Denom: ", sum_denom)
    #   #draw_crewmate_probabilities_matrix(grid)
    #   print("prob_beep_values: ")
    #   for value in prob_beep_values:
    #     print(value)
    #   print("denom_values: ")
    #   for value in denom_values:
    #     print(value)
    #   print("###################################")
    # finding probabilitiy for every combination j and k
    for rowj in grid:
      for cellj in rowj:
        if cellj.open:
          if cellj != bot_cell:
            d = distance_map[(bot_cell.x,bot_cell.y)][(cellj.x, cellj.y)]
            prob_beep_from_cellj = Decimal(math.exp((-1)*alpha*(d-1)))
            for rowk in grid:
              for cellk in rowk:
                if cellk.open:
                  if cellk != cellj:
                    if cellk != bot_cell:
                      d = distance_map[(bot_cell.x,bot_cell.y)][(cellk.x, cellk.y)]
                      prob_beep_from_cellk = Decimal(math.exp((-1)*alpha*(d-1)))
                      prob_beep_from_cellj_and_cellk = Decimal(1-(1-prob_beep_from_cellj)*(1-prob_beep_from_cellk))
                      #print("Adding values to crewmate pairs: ", (get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk)/sum_denom)
                      add_coordinates_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y, Decimal((get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk)/sum_denom))
    # finding probability for cell j
    for rowj in grid:
      for cellj in rowj:
        if cellj.open:
          if cellj != bot_cell:
            cellj_crewmate_probs = []
            for rowk in grid:
              for cellk in rowk:
                if cellk.open:
                  if cellk != cellj:
                    if cellk != bot_cell:
                      cellj_crewmate_probs.append(get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
            cellj.crewmate_probability = sum(cellj_crewmate_probs)

  if not crewmate_beep:
    denom_values = []
    #prob_beep_values = []
    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            dj = distance_map[(bot_cell.x,bot_cell.y)][(cellj.x, cellj.y)]
            prob_beep_from_cellj = Decimal((math.exp((-1)*alpha*(dj-1))))
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellk != cellj:
                    if cellk != bot_cell:
                      dk = distance_map[(bot_cell.x,bot_cell.y)][(cellk.x, cellk.y)]
                      prob_beep_from_cellk = Decimal((math.exp((-1)*alpha*(dk-1))))

                      prob_beep_from_cellj_and_cellk = Decimal((1-prob_beep_from_cellj)*(1-prob_beep_from_cellk))
                      # if prob_beep_from_cellj_and_cellk < 0:
                      #   print("prob_beep less than 0")
                      #   print(prob_beep_from_cellj_and_cellk)
                      #   print("prob_bep_cellj: ", prob_beep_from_cellj)
                      #   print("dj: ", dj)
                      #   print("prob_bep_cellk: ", prob_beep_from_cellk)
                      #   print("dk: ", dk)
                      #   print("cellj: ", cellj.loc)
                      #   print("cellk: ", cellk.loc)
                      #   print("bot_cell: ", bot_cell.loc)
                      #   draw_ship_df(grid)
                      #   draw_crewmate_probabilities_matrix(grid)
                      # for key,val in crewmate_pairs_probs.items():          #Fix
                      #   if crewmate_pairs_probs[key] == 0:
                      #     crewmate_pairs_probs[key]=0.0000001
                      denom_values.append(Decimal(get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk))
                      # if get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk <= 0:
                      #   print("get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk: ", get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk)
                      #   print("get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y): ", get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
                      #   print("denom_value less than 0")
                      #   print(prob_beep_from_cellj_and_cellk)
                      #   print("prob_bep_cellj: ", prob_beep_from_cellj)
                      #   print("dj: ", dj)
                      #   print("prob_bep_cellk: ", prob_beep_from_cellk)
                      #   print("dk: ", dk)
                      #   print("cellj: ", cellj.loc)
                      #   print("cellk: ", cellk.loc)
                      #   print("bot_cell: ", bot_cell.loc)
                      #   draw_ship_df(grid)
                      #   draw_crewmate_probabilities_matrix(grid)
                      #   print("------------------------------------------------")
                      # prob_beep_values.append(prob_beep_from_cellj_and_cellk)
                      #denom_values.append(get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
    sum_denom = sum(denom_values)
    # if sum_denom == 0:
    #   print("###################################")
    #   print("Divide by Zero Error:")
    #   print("Sum_Denom: ", sum_denom)
    #   #draw_crewmate_probabilities_matrix(grid)
    #   print("prob_beep_values: ")
    #   for value in prob_beep_values:
    #     print(value)
    #   print("denom_values: ")
    #   for value in denom_values:
    #     print(value)
    #   print("###################################")

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            d = distance_map[(bot_cell.x,bot_cell.y)][(cellj.x, cellj.y)]
            prob_beep_from_cellj = Decimal((math.exp((-1)*alpha*(d-1))))
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellk != cellj:
                    if cellk != bot_cell:
                      d = distance_map[(bot_cell.x,bot_cell.y)][(cellk.x, cellk.y)]
                      prob_beep_from_cellk = Decimal((math.exp((-1)*alpha*(d-1))))

                      prob_beep_from_cellj_and_cellk = Decimal((1-prob_beep_from_cellj)*(1-prob_beep_from_cellk))

                      add_coordinates_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y, Decimal((get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_beep_from_cellj_and_cellk)/sum_denom))
                    # add_coordinates_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y, (get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))/sum_denom)

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            cellj_crewmate_probs = []
            for row in grid:
              for cellk in row:
                if cellk != cellj:
                  if cellk.open:
                    if cellk != bot_cell:
                      cellj_crewmate_probs.append(get_value_crewmate_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
            cellj.crewmate_probability = sum(cellj_crewmate_probs)




In [20]:
# Function to add a key-value pair to the hashmap
# The key is a pair of coordinate pairs ((x, y), (a, b))
def add_coordinates_alien_pairs(x, y, a, b, value):
    alien_pairs_probs[((x, y), (a, b))] = value

# Function to get a value from the hashmap given a key
def get_value_alien_pairs(x, y, a, b):
    num_open_cells = len(detect_open_cells(grid))
    #return alien_pairs_probs.get(((x, y), (a, b)), "Coord not found")
    return alien_pairs_probs.get(((x, y), (a, b)), 1/((num_open_cells-1)*(num_open_cells-1)))


In [21]:
def initialize_alien_pairs_probs(alien_pairs_probs, grid):
  grid = initialize_detected_cells(grid, bot_cell, detector_size)
  open_cells = []
  for row in grid:
    for cell in row:
      if (cell.open):
        if not cell.cell_in_detector:
          # print("OPEN CELL OUTSIDE DET GRID",[cell.x,cell.y])
          open_cells.append(cell)

  num_open_cells = len(open_cells)
  prob = 1/((num_open_cells-1)*(num_open_cells-1))
  for row in grid:
    for cellj in row:
      if cellj.open:
        if not cellj.cell_in_detector:
          for row in grid:
            for cellk in row:
              if cellk.open:
                if not cellk.cell_in_detector:
                  if cellk != cellj:
                    add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, prob)
  for row in grid:
    for cellj in row:
      if cellj.open:
        if cellj.cell_in_detector:
          for row in grid:
            for cellk in row:
              if cellk.open:
                if cellk.cell_in_detector:
                  if cellk != cellj:
                    add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, 0)

In [22]:
def prob_alien_detected(bot_cell, detector_size, cellj, cellk) :
  prob_beep = 0
  if (bot_cell.x - detector_size) <= cellj.x and cellj.x <= (bot_cell.x + detector_size) :
    if (bot_cell.y - detector_size) <= cellj.y and cellj.y <= (bot_cell.y + detector_size):
      prob_beep = 1
  if (bot_cell.x - detector_size) <= cellk.x and cellk.x <= (bot_cell.x + detector_size) :
    if (bot_cell.y - detector_size) <= cellk.y and cellk.y <= (bot_cell.y + detector_size):
      prob_beep = 1


  return prob_beep

In [23]:
def detect_alien_pairs_probs(grid, alien_beep):
  if alien_beep:
    # for row in grid:
    #   for cellj in row:
    #     if cellj.open:
    #       for row in grid:
    #         for cellk in row:
    #           if cellk.open:
    #             if cellj != cellk:
    #               if not cellj.cell_in_detector and not cellk.cell_in_detector:
    #                 add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, 0)

    denom_values = []
    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      prob_alien_detected_given_cellj_and_cellk = prob_alien_detected(bot_cell, detector_size, cellj, cellk)
                      #print("prob_alien_detected_given_cellj_and_cellk: ", prob_alien_detected_given_cellj_and_cellk)
                      if get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_alien_detected_given_cellj_and_cellk==0:
                        cell.alien_probability=0.00000001
                      denom_values.append(get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_alien_detected_given_cellj_and_cellk)
    sum_denom_values = sum(denom_values)

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      prob_alien_detected_given_cellj_and_cellk = prob_alien_detected(bot_cell, detector_size, cellj, cellk)
                      add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, ((get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y))*prob_alien_detected_given_cellj_and_cellk)/sum_denom_values)

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            cellj_alien_probs = []
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      cellj_alien_probs.append(get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
            cellj.alien_probability = sum(cellj_alien_probs)

  if not alien_beep:
    # for row in grid:
    #   for cellj in row:
    #     if cellj.open:
    #       for row in grid:
    #         for cellk in row:
    #           if cellk.open:
    #             if  cellj.cell_in_detector or cellk.cell_in_detector:
    #               add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, 0)

    denom_values = []
    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      prob_alien_detected_given_cellj_and_cellk = prob_alien_detected(bot_cell, detector_size, cellj, cellk)
                      #print("prob_alien_detected_given_cellj_and_cellk: ", prob_alien_detected_given_cellj_and_cellk)
                      if get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_alien_detected_given_cellj_and_cellk==0:
                        cell.alien_probability=0.00000001
                      denom_values.append(get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y)*prob_alien_detected_given_cellj_and_cellk)
    sum_denom_values = sum(denom_values)

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      prob_alien_detected_given_cellj_and_cellk = prob_alien_detected(bot_cell, detector_size, cellj, cellk)
                      add_coordinates_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y, ((get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y))*prob_alien_detected_given_cellj_and_cellk)/sum_denom_values)

    for row in grid:
      for cellj in row:
        if cellj.open:
          if cellj != bot_cell:
            cellj_alien_probs = []
            for row in grid:
              for cellk in row:
                if cellk.open:
                  if cellj != cellk:
                    if cellk != bot_cell:
                      cellj_alien_probs.append(get_value_alien_pairs(cellj.x, cellj.y, cellk.x, cellk.y))
            cellj.alien_probability = sum(cellj_alien_probs)

In [24]:
def alien_pairs_move_probs(grid):
  denom_values = []
  for row in grid:
    for cellj in row:
      if cellj.open:
        if cellj != bot_cell:
          for row in grid:
            for cellk in row:
              if cellk.open:
                if cellj != cellk:
                  if cellj != bot_cell:
                    for neighborj in cellj.neighbors:
                      for neighbork in cellk.neighbors:
                        if neighborj != neighbork:
                          denom_values.append(get_value_alien_pairs(neighborj.x, neighborj.y, neighbork.x, neighbork.y)*(neighborj.prob_alien_moved_from*neighbork.prob_alien_moved_from))
  sum_denom_values = sum(denom_values)
  beta_t = 1/sum_denom_values

  for row in grid:
    for cellj in row:
      if cellj.open:
        if cellj != bot_cell:
          x_prime_values = []
          for row in grid:
            for cellk in row:
              if cellk.open:
                if cellj != cellk:
                  if cellk != bot_cell:
                    for neighborj in cellj.neighbors:
                      for neighbork in cellk.neighbors:
                        if neighborj != neighbork:
                          x_prime_values.append(get_value_alien_pairs(neighborj.x, neighborj.y, neighbork.x, neighbork.y)*(neighborj.prob_alien_moved_from*neighbork.prob_alien_moved_from))
          x_prime = sum(x_prime_values)
          cellj.alien_probability = beta_t*x_prime

In [25]:
import pandas as pd
def draw_alien_probabilities_matrix(grid):
  columns_123=[]
  rows_123=[]
  for i in range(len(grid)):
    columns_123.append(i)
    rows_123.append(i)

  values_matrix=[]

  for row in grid:
    temp_row=[]
    for cell in row:
      if cell.value == 'B':
        temp_row.append("B")
      elif cell.alien_probability==0 and cell.open!=True:
        temp_row.append('X')
      else:
        temp_row.append(cell.alien_probability)
    values_matrix.append(temp_row)

  df = pd.DataFrame(values_matrix, columns=columns_123, index=rows_123)
  print(df.to_string())
  print("\n")

In [26]:
import pandas as pd
def draw_crewmate_probabilities_matrix(grid):
    # Set pandas option to display more floating point digits (e.g., 15 digits)
    pd.set_option('display.float_format', '{:.15f}'.format)

    columns_123 = []
    rows_123 = []
    for i in range(len(grid)):
        columns_123.append(i)
        rows_123.append(i)

    values_matrix = []

    for row in grid:
        temp_row = []
        for cell in row:
            if cell.value == 'B':
                temp_row.append("B")
            elif not cell.open:
                temp_row.append('X')
            else:
                temp_row.append(cell.crewmate_probability)

        values_matrix.append(temp_row)

    df = pd.DataFrame(values_matrix, columns=columns_123, index=rows_123)
    print(df.to_string())
    print("\n")

    # Reset the display option to default if needed elsewhere
    pd.reset_option('display.float_format')


In [27]:
def sum_probabilities_alien(grid):
  probabilities = []
  for row in grid:
    for cell in row:
      probabilities.append(cell.alien_probability)
  return sum(probabilities)

In [28]:
def sum_probabilities_crewmate(grid):
  probabilities = []
  for row in grid:
    for cell in row:
      probabilities.append(cell.crewmate_probability)
  return sum(probabilities)

In [29]:
def detect_open_neighbors_bot_1(current_cell, matrix_size):
  open_neighbors = []
  arr=[current_cell.left,current_cell.right,current_cell.top,current_cell.bottom]
  for i in arr:
    if (i==None):
      continue
    if (i.x==-1 or i.y==-1 or i.x==matrix_size or i.y==matrix_size):
      continue
    if i.value==0:
      continue
    else:
      open_neighbors.append(i)
  return open_neighbors

In [30]:
def sort_open_neighbors(grid, open_neighs):
  open_neighs.sort(key=lambda cell: cell.crewmate_probability, reverse=True)
  on_copy=open_neighs
  open_neighbors_with_alien_prob_0 = [cell for cell in open_neighs if cell.alien_probability < 0.2]
  if (len(open_neighbors_with_alien_prob_0)==0):
    open_neighs=on_copy
    sorted_open_neighbors = sorted(open_neighs, key=lambda obj: (-obj.crewmate_probability, obj.alien_probability))
  else:
    return open_neighbors_with_alien_prob_0
  return sorted_open_neighbors

In [31]:
def find_path(source,dest):
  src=(source[0],source[1])
  dst=(dest[0],dest[1])
  tempo=pred_map[src][dst]
  if tempo is None:
    #print(f"The destination {dst} is a blocked cell")
    #print(pred_map[src])
    return []
  found_path=[[tempo[0],tempo[1]],[dest[0],dest[1]]]
  while tempo!=src:
    tempo=pred_map[src][tempo]
    found_path.insert(0,[tempo[0],tempo[1]])
  return found_path

In [32]:
def move_bot1(grid, bot_cell, path, aliens, crewmates, time_steps):
  if bot_cell.value=='A' or bot_cell.value=='C*':
    return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
  while True:
    next=path[1]
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    if next_cell.alien_probability>0.00001:
      #print("condition entered")
      status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
      max_prob_val=max(c.crewmate_probability for r in grid for c in r)
      max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
      max_prob_cell=random.choice(max_prob_index)
      path=find_path(bot_cell.loc,max_prob_cell)
      continue
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1
    if (grid[next[0]][next[1]].value!='C' and grid[next[0]][next[1]].value!='C*' and grid[next[0]][next[1]].value!='A'):
      grid[next[0]][next[1]].value='B'
      next_cell.value='B'
      next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    if bot_cell.value=='A' or bot_cell.value=='C*':
      #draw_ship_df(grid)
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    if bot_cell.value=='C':
      # grid[next[0]][next[1]].value='B'
      # next_cell.value='B'
      bot_cell.value='B'
      #draw_ship_df(grid)
      return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)

    bot_cell.crewmate_probability=0
    #print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("AFTER TAKE STEP")
    #draw_ship_df(grid)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    if bot_cell.value=='A' or bot_cell.value=='C*':
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #print("PATH2  RETURNED BY BFS IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

Bot 1 Driver Code

In [ ]:
# from numpy.core.fromnumeric import take
bot_id = 1
size = 10
num_aliens = 1
num_crew = 1
detector_size_values = [1, 5, 7, 10, 14]

alpha_01_percent = -2*math.log(.01)/(size/2)
alpha_10_percent = -2*math.log(.1)/(size/2)
alpha_25_percent = -2*math.log(.25)/(size/2)
alpha_50_percent = -2*math.log(.5)/(size/2)
alpha_75_percent = -2*math.log(.75)/(size/2)
alpha_99_percent = -2*math.log(.99)/(size/2)

alpha_values = [alpha_25_percent, alpha_50_percent]
crewmates = []
num_time_steps=[]
sim=0
successes=0
alien_caught=0
num_sims= 5
results = []
for alpha in alpha_values:
  for detector_size in detector_size_values:
    num_time_steps=[]
    sim=0
    crew1_res=0
    crew_res_both=0
    alien_caught=0
    for _ in range(num_sims):
      alien_beep = False
      time_steps=1
      grid = create_grid(size, size)
      for row in grid:
        for cell in row:
          cell.neighbors = get_neighbors(cell, grid)
      # bot_cell = create_bot(grid)
      aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

      grid=initialize_cell_neighbors(grid)
      num_open_cells = len(detect_open_cells(grid))
      for row in grid:
        for cell in row:
          cell.open = detect_if_cell_open(cell)
          cell.neighbors = get_open_neighbors(cell, grid)
      distance_map={}
      pred_map={}
      for i in range(len(grid)):
        for j in range(len(grid[0])):
          s=(i,j)
          dlist,f=dijkstra(grid,s)
          distance_map[s]=dlist
          pred_map[s]=f

      #initialize_prob_alien(grid, num_aliens)
      initialize_prob_crewmate(grid, num_crew)
      initialize_prob_alien_moved(grid)

      # bot_cell = create_bot(grid)
      # print("INITIAL SHIP")
      # draw_ship_df(grid)
      # print("INITIAL CREW PROBS MATRIX")
      # draw_crewmate_probabilities_matrix(grid)
      # print("initial alien prob matrix")
      # draw_alien_probabilities_matrix(grid)
      # print("Sum of Alien Probabilities")
      # print(sum_probabilities_alien(grid))
      # print("Sum of Crewmate Probabilities")
      # print(sum_probabilities_crewmate(grid))
      max_prob_val=max(c.crewmate_probability for r in grid for c in r)
      max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
      max_prob_cell=random.choice(max_prob_index)
      path=find_path(bot_cell.loc,max_prob_cell)
      # print("-----------------------------------------------------------------------------------------------------")
      # print("crew member was "+res+" via path",path)
      # if len(path)!=0:
      #   print("Now moving Bot to rescue Crewmate...")

      msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot1(grid, bot_cell, path, aliens, crewmates, time_steps)
      #print(msg)

      # print("The time steps taken are: ",time_steps)
      #print("TIME TIME TIME",time_steps)
      sim+=1
      #print("Simmulation Number: ", sim)
      if (msg=="The bot was caught by the aliens!"):
        alien_caught+=1
      elif (msg=="The bot has rescued a crewmate"):
        successes+=1
        num_time_steps.append(time_steps)


    # print("OUT OF",num_sims,"SIMULATIONS")
    # print("BOT RESCUED CREWMATE",successes,"TIMES")
    # print("BOT WAS CAUGHT BY ALIENS",alien_caught,"TIMES")

    # print("AVERAGE MOVES REQUIRED TO RESCUE CREWMATE IS",sum(num_time_steps)/successes)
      if crew1_res != 0:
        results.append([alpha, detector_size, sim, crew1_res, crew1_res/sim, sum(num_time_steps)/crew1_res])
      else:
        results.append([alpha, detector_size, sim, crew1_res, crew1_res/sim, 0])
      print(results)

      results_csv = pd.DataFrame(results)
      results_csv.to_csv("results_bot_1.csv")

In [ ]:
locs=grid[6][3].neighbors
for cell in locs:
  print(cell.loc)
print(get_open_neighbors(grid[6][3],grid))

In [ ]:
#testing
size=10
grid = create_grid(size, size)
for row in grid:
  for cell in row:
    cell.neighbors = get_neighbors(cell, grid)
# bot_cell = create_bot(grid)
aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

grid=initialize_cell_neighbors(grid)
draw_ship_df(grid)
for row in grid:
  for cell in row:
    cell.open = detect_if_cell_open(cell)
    cell.neighbors = get_open_neighbors(cell, grid)
distance_map={}
pred_map={}
for i in range(len(grid)):
  for j in range(len(grid[0])):
    s=(i,j)
    dlist,f=dijkstra(grid,s)
    distance_map[s]=dlist
    pred_map[s]=f

testpath=find_path(s,(1,2))
print(testpath)
print(pred_map)

In [ ]:
print(pred_map[(3,8)])

Bot 2 Code

In [ ]:
def find_path_bot2(source,dest_list):
  len_list=[]
  pth=[]
  for i,ele in enumerate(dest_list):
    pth.insert(i,find_path(source,ele))
    if len(pth[i])==0:
      len_list.insert(i,float('inf'))
      continue
    len_list.insert(i,len(pth[i]))
  min_len=min(len_list)
  #print("min length is: ",min_len)
  #print("len list is: ",len_list)
  for i,p in enumerate(pth):
    if len_list[i]==min_len:
      return pth[i]


In [ ]:
def move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep):
  while True:
    bot_cell.crewmate_probability=0

    if alien_beep:
      #print("ALIEN AVOIDANCE ACTIVATED")
      alien_avoidance(grid)
      #print(path)
    #if bot_cell.value=='C':
    #print("Top of move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("crew_cell", crewmates[0].loc)
    # if bot_cell == crewmates[0]:
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
          # grid[next[0]][next[1]].value='B'
          # next_cell.value='B'
          bot_cell.value='B'
          #draw_ship_df(grid)
          return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps
    #if bot_cell.value=='A' or bot_cell.value=='C*':
    #print("line 15 move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("alien_cell", aliens[0].loc)
    for alien_cell in aliens:
      if bot_cell.value== 'A':
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    curr_v=0
    while path[curr_v]!=bot_cell.loc:
      curr_v+=1
    curr_v+=1
    if curr_v==len(path):
      #print("end of path reached")
      bot_cell.crewmate_probability=0
      max_prob_val=max(c.crewmate_probability for r in grid for c in r)             # Find max crew prob
      max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]   #Find cell of max prob
      #print("closest_max_prob_cell: ", max_prob_index)
      #draw_crewmate_probabilities_matrix(grid)
      path = find_path_bot2(bot_cell.loc, max_prob_index)                  # Find closest cell with max prob
      #print("bot_cell: ",bot_cell.loc)
      #print(path)
      msg, bot_cell, crewmates, grid, aliens, time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path
      #msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot1(grid, bot_cell, path, aliens, crewmates, time_steps)    #pathing to max crew prob cell
      #return "reached centre", bot_cell, crewmates, grid, aliens,time_steps
    #print("line 37 move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("alien_cell", aliens[0].loc)
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        #draw_ship_df(grid)
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #if bot_cell.value=='C':
    #print("line 44 move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("crew_cell", crewmates[0].loc)
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
        # grid[next[0]][next[1]].value='B'
        # next_cell.value='B'
        bot_cell.value='B'
        #draw_ship_df(grid)
        return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps
    next=path[curr_v]
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1

    for crew_cell in crewmates:
      if (grid[next[0]][next[1]] != crew_cell):
        for alien_cell in aliens:
          if (grid[next[0]][next[1]] != alien_cell):
            grid[next[0]][next[1]].value='B'
            next_cell.value='B'
            next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    #if bot_cell.value=='A' or bot_cell.value=='C*':
    #print("line 68 move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("alien_cell", aliens[0].loc)
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        #draw_ship_df(grid)
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #if bot_cell.value=='C':
    #print("line 75 move_bot2")
    #print("bot_cell", bot_cell.loc)
    #print("crew_cell", crewmates[0].loc)
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
        # grid[next[0]][next[1]].value='B'
        # next_cell.value='B'
        bot_cell.value='B'
        #draw_ship_df(grid)
        return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)
    # print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens, time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("Aliens post take_step: ", aliens)
    #print("AFTER TAKE STEP")

    #print("alien_beep within move bot2", alien_beep)
    #draw_ship_df(grid)
    #draw_crewmate_probabilities_matrix(grid)

    if curr_v != len(path):
      max_crew_prob_cell = random.choice(detect_open_cells(grid))
      for row in grid:
        for cell in row:
          if cell.crewmate_probability > max_crew_prob_cell.crewmate_probability:
            max_crew_prob_cell = cell
      if max_crew_prob_cell.crewmate_probability != path[len(path)-1] and max_crew_prob_cell.crewmate_probability > 0.5:                                                # Exiting path to center if a cells crewmate prob goes over .5
        #print("detected high prob crewmate")
        bot_cell.crewmate_probability=0
        max_prob_val=max(c.crewmate_probability for r in grid for c in r)             # Find max crew prob
        max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]   #Find cell of max prob
        #print("closest_max_prob_cell: ", max_prob_index)
        #draw_crewmate_probabilities_matrix(grid)
        path = find_path_bot2(bot_cell.loc, max_prob_index)                  # Find closest cell with max prob
        #print("bot_cell: ",bot_cell.loc)
        #print(path)
        msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path

        # return "entered a high prob region", bot_cell, crewmates, grid, aliens,time_steps
    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #print("PATH2  RETURNED BY BFS IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  #print("End of move_bot2")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

In [ ]:
def alien_avoidance(grid):
  left_sector = 0
  right_sector = 0
  top_sector = 0
  bottom_sector = 0
  for x in range(len(grid)):
    for y in range(len(grid[0])):
      if x < y and x + y < len(grid):
        right_sector = right_sector + grid[x][y].alien_probability
      if x <= y and x + y >= len(grid):
        top_sector = top_sector + grid[x][y].alien_probability
      if x >= y and x + y <= len(grid):
        bottom_sector = bottom_sector + grid[x][y].alien_probability
      if x >= y and x + y >= len(grid):
        left_sector = left_sector + grid[x][y].alien_probability
  max_sector = max(left_sector, right_sector, top_sector, bottom_sector)
  if max_sector == left_sector:
    #print("Max alien prob is left sector")
    new_x, new_y = bot_cell.x + 1, bot_cell.y
    if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x+1][bot_cell.y].open:
      path = [bot_cell.loc, grid[bot_cell.x+1][bot_cell.y].loc]
    else:
      if min(top_sector, bottom_sector) == top_sector:
        new_x, new_y = bot_cell.x, bot_cell.y + 1
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y+1].open:
          path = [bot_cell.loc, grid[bot_cell.x][bot_cell.y+1].loc]
      else:
        new_x, new_y = bot_cell.x, bot_cell.y - 1
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y-1].open:
          path = [bot_cell.loc,grid[bot_cell.x][bot_cell.y-1].loc]
  if max_sector == right_sector:
    #print("Max alien prob is right sector")
    new_x, new_y = bot_cell.x - 1, bot_cell.y
    if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x-1][bot_cell.y].open:
      path = [bot_cell.loc,grid[bot_cell.x-1][bot_cell.y].loc]
    else:
      if min(top_sector, bottom_sector) == top_sector:
        new_x, new_y = bot_cell.x, bot_cell.y + 1
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y+1].open:
          path = [bot_cell.loc,grid[bot_cell.x][bot_cell.y+1].loc]
      else:
        new_x, new_y = bot_cell.x, bot_cell.y - 1
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y-1].open:
          path = [bot_cell.loc,grid[bot_cell.x][bot_cell.y-1].loc]
  if max_sector == top_sector:
    #print("Max alien prob is top sector")
    new_x, new_y = bot_cell.x, bot_cell.y-1
    if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y-1].open:
      path = [bot_cell.loc,grid[bot_cell.x][bot_cell.y-1].loc]
    else:
      if min(right_sector, left_sector) == left_sector:
        new_x, new_y = bot_cell.x-1, bot_cell.y
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x-1][bot_cell.y].open:
          path = [bot_cell.loc,grid[bot_cell.x-1][bot_cell.y].loc]
      else:
        new_x, new_y = bot_cell.x+1, bot_cell.y
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x+1][bot_cell.y].open:
          path = [bot_cell.loc,grid[bot_cell.x+1][bot_cell.y].loc]
  if max_sector == bottom_sector:
    #print("Max alien prob is bottom sector")
    new_x, new_y = bot_cell.x, bot_cell.y + 1
    if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x][bot_cell.y+1].open:
      path = [bot_cell.loc,grid[bot_cell.x][bot_cell.y+1].loc]
    else:
      if min(right_sector, left_sector) == left_sector:
        new_x, new_y = bot_cell.x-1, bot_cell.y
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x-1][bot_cell.y].open:
          path = [bot_cell.loc,grid[bot_cell.x-1][bot_cell.y].loc]
      else:
        new_x, new_y = bot_cell.x+1, bot_cell.y
        if is_valid(Cell(new_x, new_y, '', 3), grid) and grid[bot_cell.x+1][bot_cell.y].open:
          path = [bot_cell.loc,grid[bot_cell.x+1][bot_cell.y].loc]

In [ ]:
bot_id=2
size=5
num_aliens = 1
num_crew = 1
detector_size = 1
alpha = -2*math.log(.1)/size                                                                  # Caluclating alpha based on ship size
print("Alpha: ", alpha)
num_time_steps=[]
sim=0
successes=0
alien_caught=0
for _ in range(100):
  #alien_beep = False
  crewmates = []
  time_steps=1
  grid = create_grid(size, size)
  for row in grid:
    for cell in row:
      cell.neighbors = get_neighbors(cell, grid)
  aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)
  print("Crewmates location: ")
  print(crewmates[0].loc)
  print("Aliens location: ")
  print(aliens[0].loc)
  grid=initialize_cell_neighbors(grid)
  num_open_cells = len(detect_open_cells(grid))
  for row in grid:
    for cell in row:
      cell.open = detect_if_cell_open(cell)
      cell.neighbors = get_open_neighbors(cell, grid)
  distance_map={}
  pred_map={}
  for i in range(len(grid)):
    for j in range(len(grid[0])):
      s=(i,j)
      dlist,f=dijkstra(grid,s)
      distance_map[s]=dlist
      pred_map[s]=f
  initialize_prob_crewmate(grid, num_crew)
  initialize_prob_alien_moved(grid)
  print("INITIAL SHIP")
  draw_ship_df(grid)
  print("INITIAL CREW PROBS MATRIX")
  draw_crewmate_probabilities_matrix(grid)
  print("initial alien prob matrix")
  draw_alien_probabilities_matrix(grid)
  print("Sum of Alien Probabilities")
  print(sum_probabilities_alien(grid))
  print("Sum of Crewmate Probabilities")
  print(sum_probabilities_crewmate(grid))
  #------------------------------
  centre_cells=[]
  if len(grid)%2==1:
    centre_cells.append([len(grid)//2,len(grid)//2])                             # Finding Center Cell
  else:
    centre_cells.append([len(grid)//2,len(grid)//2])
    centre_cells.append([len(grid)//2-1,len(grid)//2])
    centre_cells.append([len(grid)//2,len(grid)//2-1])
    centre_cells.append([len(grid)//2-1,len(grid)//2-1])
  if len(centre_cells)==1 and grid[centre_cells[0][0]][centre_cells[0][1]].open==False:     #Corner case closed center cell
    centre_cells_mod=grid[centre_cells[0][0]][centre_cells[0][1]].neighbors
    centre_cells=[ce.loc for ce in centre_cells_mod]
  print("BOT CELL ", bot_cell.loc)
  print("CENTER CELL ", centre_cells[0])
  if bot_cell.loc == centre_cells[0]:                                                 # Corner case where bot spawns on center cell
    print("centre cells: ",centre_cells)
    print("bot_cell: ",bot_cell.loc)
    print("bot_cell spawned at center cell")
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)             # Find max crew prob
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]   #Find cell of max prob
    print("closest_max_prob_cell: ", max_prob_index)
    draw_crewmate_probabilities_matrix(grid)
    path = find_path_bot2(bot_cell.loc, max_prob_index)                  # Find closest cell with max prob
    print("bot_cell: ",bot_cell.loc)
    print(path)
    msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path

  path = find_path_bot2(bot_cell.loc, centre_cells)                                 # Path to center cell
  print("centre cells: ",centre_cells)
  print("bot_cell: ",bot_cell.loc)
  print(path)
  msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path
  if msg=="reached centre":
    print("REACHED THE CENTRE")
  elif msg=="entered a high prob region":
    print("REACHED AN AREA WITH PROB >0.25")
  #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)

  print(msg)
  sim+=1
  print("Simmulation Number: ", sim)
  if (msg=="The bot was caught by the aliens!"):
      alien_caught+=1
  elif (msg=="The bot has rescued a crewmate"):
      successes+=1
      num_time_steps.append(time_steps)

print("OUT OF 30 SIMULATIONS")
print("BOT RESCUED CREWMATE",successes,"TIMES")
print("BOT WAS CAUGHT BY ALIENS",alien_caught,"TIMES")

print("AVERAGE MOVES REQUIRED TO RESCUE CREWMATE IS",sum(num_time_steps)/successes)
print("AVERAGE NUMBER OF CREWMATES RESCUED PER SIMULATION",successes/30)
print("The time steps taken are: ",time_steps)
print("TIME TIME TIME",time_steps)



BOT 3 CODE

In [ ]:
def move_bot3(grid, bot_cell, path, aliens, crewmates, time_steps):
  #print("NORmAl")
  for alien_cell in aliens:
    if bot_cell == alien_cell:
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
  while len(path) == 0:
    print("NO PATH NOW SO ALIENS MOVE")
    status,crewmates, aliens,time_steps = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
  # for next in path[1::]:
  while True:
    next=path[1]
    print("MOVING THE BOT TO",next)
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1
    if (grid[next[0]][next[1]].value!='C' and grid[next[0]][next[1]].value!='C*' and grid[next[0]][next[1]].value!='A'):
      grid[next[0]][next[1]].value='B'
      next_cell.value='B'
      next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        print("ALIENS CAUGHT BOT AT",bot_cell.loc)
        draw_ship_df(grid)
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
        # grid[next[0]][next[1]].value='B'
        # next_cell.value='B'
        print("BOT FOUND CREWMATE AT",bot_cell.loc)
        draw_ship_df(grid)
        bot_cell.value='B'
        #draw_ship_df(grid)
        return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)

    bot_cell.crewmate_probability=0

    #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    # print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    print("BOT MOVED")
    draw_ship_df(grid)

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("AFTER TAKE STEP")

    print("ALIEN MOVED")
    draw_ship_df(grid)

    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    print("PATH GOING TO BE FOLLOWED BY BOT3 IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

In [ ]:
# BOT 3 DRIVER CODE

bot_id = 3
size = 5
num_aliens = 1
num_crew = 2
detector_size = 1
alpha = -2*math.log(.1)/size                                                                  # Caluclating alpha based on ship size
print("Alpha: ", alpha)
crewmate_pairs_probs = {}
crewmates = []
num_time_steps=[]
sim=0
crew1_res=0
crew_res_both=0
alien_caught=0

for _ in range(100):
  alien_beep = False
  time_steps=1
  crewmate_pairs_probs = {}
  crewmates = []
  grid = create_grid(size, size)
  for row in grid:
    for cell in row:
      cell.neighbors = get_neighbors(cell, grid)
  # bot_cell = create_bot(grid)
  aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

  # crew_start_cell = random.choice(detect_open_cells(grid))
  # if (crew_start_cell.x==bot_cell.x and crew_start_cell.y==bot_cell.y):
  #   crew_start_cell = random.choice(detect_open_cells(grid))
  # if (crew_start_cell.x==crew_cell.x and crew_start_cell.y==crew_cell.y):
  #   crew_start_cell = random.choice(detect_open_cells(grid))
  # while(crew_start_cell==bot_cell or crew_start_cell==crew_cell):
  #   crew_start_cell = random.choice(detect_open_cells(grid))
  # crew_start_cell.value = 'C'

  grid=initialize_cell_neighbors(grid)
  num_open_cells = len(detect_open_cells(grid))
  for row in grid:
    for cell in row:
      cell.open = detect_if_cell_open(cell)
      cell.neighbors = get_open_neighbors(cell, grid)
  # initialize_prob_alien(grid, num_aliens)

  distance_map={}
  pred_map={}
  for i in range(len(grid)):
    for j in range(len(grid[0])):
      s=(i,j)
      dlist,f=dijkstra(grid,s)
      distance_map[s]=dlist
      pred_map[s]=f

  # initialize_crewmate_pairs_probs(crewmate_pairs_probs)
  # print(crewmate_pairs_probs)
  initialize_prob_crewmate(grid, num_crew)
  initialize_prob_alien_moved(grid)
  # bot_cell = create_bot(grid)
  draw_ship_df(grid)
  print("INITIAL CREW PROBS MATRIX")
  draw_crewmate_probabilities_matrix(grid)
  print("initial alien prob matrix")
  draw_alien_probabilities_matrix(grid)
  print("Sum of Alien Probabilities")
  print(sum_probabilities_alien(grid))
  print("Sum of Crewmate Probabilities")
  print(sum_probabilities_crewmate(grid))

  # for r in grid:
  #   for c in r:
  #     for r2 in grid:
  #       for c2 in r2:
  #         print("PAIRWISE PROB FOR",c.loc,"AND",c2.loc,"IS",get_value_crewmate_pairs(c.x,c.y,c2.x,c2.y))

  #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
  max_prob_val=max(c.crewmate_probability for r in grid for c in r)
  max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
  max_prob_cell=random.choice(max_prob_index)
  path=find_path(bot_cell.loc,max_prob_cell)
  for row in grid:
    for cell in row:
      cell.visited=False
      cell.parent=None

  print("-----------------------------------------------------------------------------------------------------")
  print("crew member was found via path",path)
  if len(path)!=0:
    print("Now moving Bot to rescue Crewmate...")

  msg1, bot_cell, crewmates, grid, aliens,time_steps = move_bot3(grid, bot_cell, path, aliens, crewmates, time_steps)
  print(msg1)

  if (msg1=="The bot has rescued a crewmate"):
    print("NEW STARTING POSITION OF BOT IS",bot_cell.loc)

    bot_cell.crewmate_probability=0

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[bot_cell.x][bot_cell.y].value='B'
    bot_cell.value='B'

    draw_ship_df(grid)

    #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    print("-----------------------------------------------------------------------------------------------------")
    print(" Second crew member was found via path",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot3(grid, bot_cell, path, aliens, crewmates, time_steps)
    print(msg2)

  if (msg1=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
    alien_caught+=1
  if (msg1=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
    crew1_res+=1
  if (msg1=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
    crew_res_both+=1
    num_time_steps.append(time_steps)

  print("END OF SIMULATION. RELOAD GRID.")
  print("\n")
  print("-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*-*")
  print("\n")

print("OUT OF 30 SIMULATIONS AND 60 CREWMATES")
print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")

print("AVERAGE NUMBER OF CREWMATES RESCUED PER SIMULATION",(crew1_res+crew_res_both*2)/30)

if crew_res_both != 0:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
else:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESQUED")


BOT 4 Code

In [ ]:
def move_bot4(grid, bot_cell, path, aliens, crewmates, time_steps):
  #print("NORmAl")
  for alien_cell in aliens:
    if bot_cell == alien_cell:
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
  while len(path) == 0:
    #print("NO PATH NOW SO ALIENS MOVE")
    status,crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
  # for next in path[1::]:
  while True:
    next=path[1]
    #print("MOVING THE BOT TO",next)
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1
    if (grid[next[0]][next[1]].value!='C' and grid[next[0]][next[1]].value!='C*' and grid[next[0]][next[1]].value!='A'):
      grid[next[0]][next[1]].value='B'
      next_cell.value='B'
      next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        #print("ALIENS CAUGHT BOT AT",bot_cell.loc)
        #draw_ship_df(grid)
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
        # grid[next[0]][next[1]].value='B'
        # next_cell.value='B'
        #print("BOT FOUND CREWMATE AT",bot_cell.loc)
        #draw_ship_df(grid)
        bot_cell.value='B'
        #draw_ship_df(grid)
        return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)

    bot_cell.crewmate_probability=0

    #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    # print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    #print("BOT MOVED")
    #draw_ship_df(grid)

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("AFTER TAKE STEP")

    #print("ALIEN MOVED")
    #draw_ship_df(grid)

    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #print("PATH GOING TO BE FOLLOWED BY BOT4 IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

In [ ]:
# BOT 4 DRIVER CODE
bot_id = 4
size = 30
num_aliens = 1
num_crew = 2
detector_size_values = [5]
alpha_10_percent = -2*math.log(.1)/(size/2)
alpha_25_percent = -2*math.log(.25)/(size/2)
alpha_50_percent = -2*math.log(.5)/(size/2)
alpha_values = [alpha_25_percent, alpha_50_percent]
crewmate_pairs_probs = {}
crewmates = []

results = []
num_sim = 10
for alpha in alpha_values:
  for detector_size in detector_size_values:
    num_time_steps=[]
    sim=0
    crew1_res=0
    crew_res_both=0
    alien_caught=0
    for _ in range(num_sim):
      print("Sim ", sim, "started")
      alien_beep = False
      time_steps=1
      crewmate_pairs_probs = {}
      crewmates = []
      grid = create_grid(size, size)
      for row in grid:
        for cell in row:
          cell.neighbors = get_neighbors(cell, grid)
      # bot_cell = create_bot(grid)
      aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

      # crew_start_cell = random.choice(detect_open_cells(grid))
      # if (crew_start_cell.x==bot_cell.x and crew_start_cell.y==bot_cell.y):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # if (crew_start_cell.x==crew_cell.x and crew_start_cell.y==crew_cell.y):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # while(crew_start_cell==bot_cell or crew_start_cell==crew_cell):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # crew_start_cell.value = 'C'

      grid=initialize_cell_neighbors(grid)
      num_open_cells = len(detect_open_cells(grid))
      for row in grid:
        for cell in row:
          cell.open = detect_if_cell_open(cell)
          cell.neighbors = get_open_neighbors(cell, grid)
      # initialize_prob_alien(grid, num_aliens)

      distance_map={}
      pred_map={}
      for i in range(len(grid)):
        for j in range(len(grid[0])):
          s=(i,j)
          dlist,f=dijkstra(grid,s)
          distance_map[s]=dlist
          pred_map[s]=f
      #print(distance_map)

      initialize_prob_crewmate(grid, num_crew)
      initialize_crewmate_pairs_probs(crewmate_pairs_probs)
      # print(crewmate_pairs_probs)
      initialize_prob_alien_moved(grid)
      # bot_cell = create_bot(grid)
      # draw_ship_df(grid)
      # print("INITIAL CREW PROBS MATRIX")
      # draw_crewmate_probabilities_matrix(grid)
      # print("initial alien prob matrix")
      # draw_alien_probabilities_matrix(grid)
      # print("Sum of Alien Probabilities")
      # print(sum_probabilities_alien(grid))
      # print("Sum of Crewmate Probabilities")
      # print(sum_probabilities_crewmate(grid))

      # for r in grid:
      #   for c in r:
      #     for r2 in grid:
      #       for c2 in r2:
      #         print("PAIRWISE PROB FOR",c.loc,"AND",c2.loc,"IS",get_value_crewmate_pairs(c.x,c.y,c2.x,c2.y))

      #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
      max_prob_val=max(c.crewmate_probability for r in grid for c in r)
      max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
      max_prob_cell=random.choice(max_prob_index)
      path=find_path(bot_cell.loc,max_prob_cell)
      for row in grid:
        for cell in row:
          cell.visited=False
          cell.parent=None

      # print("-----------------------------------------------------------------------------------------------------")
      # print("crew member was found via path",path)
      # if len(path)!=0:
      #   print("Now moving Bot to rescue Crewmate...")

      msg1, bot_cell, crewmates, grid, aliens,time_steps = move_bot4(grid, bot_cell, path, aliens, crewmates, time_steps)
      # print(msg1)
      sim+=1
      if (msg1=="The bot has rescued a crewmate"):
        # print("NEW STARTING POSITION OF BOT IS",bot_cell.loc)

        bot_cell.crewmate_probability=0

        for row in grid:
          for cell in row:
            cell.visited=False
            cell.parent=None

        grid[bot_cell.x][bot_cell.y].value='B'
        bot_cell.value='B'

        #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
        max_prob_val=max(c.crewmate_probability for r in grid for c in r)
        max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
        max_prob_cell=random.choice(max_prob_index)
        path=find_path(bot_cell.loc,max_prob_cell)
        # print("-----------------------------------------------------------------------------------------------------")
        # print(" Second crew member was found via path",path)

        for row in grid:
          for cell in row:
            cell.visited=False
            cell.parent=None

        msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot4(grid, bot_cell, path, aliens, crewmates, time_steps)
        # print(msg2)

      if (msg1=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
        alien_caught+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
        crew1_res+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
        crew_res_both+=1
        num_time_steps.append(time_steps)

      #print("END OF SIMULATION. RELOAD GRID.")

    # print("OUT OF 30 SIMULATIONS AND 60 CREWMATES")
    # print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")
    # print("AVERAGE NUMBER OF CREWMATES RESCUED PER SIMULATION",(crew1_res+crew_res_both*2)/30)
    # if crew_res_both != 0:
    #   print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
    # else:
    #   print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESQUED")

      if crew_res_both != 0:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, sum(num_time_steps)/crew_res_both])
      else:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, 0])
      print(results)

      results_csv = pd.DataFrame(results)
      results_csv.to_csv("results_bot_4.csv")

In [ ]:
results_csv = pd.DataFrame(results)
results_csv.to_csv("results_bot_4.csv")

In [ ]:
draw_crewmate_probabilities_matrix(grid)

Bot 5 Code

In [ ]:
from google.colab import files
import io
bot_id=5
size=30
num_aliens = 1
num_crew = 2
detector_size_values = [5, 7, 10]
alpha_10_percent = -2*math.log(.1)/(size/2)
alpha_25_percent = -2*math.log(.25)/(size/2)
alpha_50_percent = -2*math.log(.5)/(size/2)
alpha_values = [alpha_25_percent, alpha_50_percent]
#print("Alpha: ", alpha)
results = []
num_sim = 3
for alpha in alpha_values:
  for detector_size in detector_size_values:
    sim=0
    crew1_res=0
    num_time_steps=[]
    crew_res_both=0
    alien_caught=0
    for _ in range(num_sim):
      print(sim)
      alien_beep = False
      crewmates = []
      crewmate_pairs_probs = {}
      alien_pairs_probs = {}
      time_steps=1
      grid = create_grid(size, size)
      for row in grid:
        for cell in row:
          cell.neighbors = get_neighbors(cell, grid)
      aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)
      #print("Crewmates location: ")
      #print(crewmates[0].loc)
      #print("Aliens location: ")
      #print(aliens[0].loc)
      grid=initialize_cell_neighbors(grid)
      num_open_cells = len(detect_open_cells(grid))
      for row in grid:
        for cell in row:
          cell.open = detect_if_cell_open(cell)
          cell.neighbors = get_open_neighbors(cell, grid)
      distance_map={}
      pred_map={}
      for i in range(len(grid)):
        for j in range(len(grid[0])):
          s=(i,j)
          dlist,f=dijkstra(grid,s)
          distance_map[s]=dlist
          pred_map[s]=f
      initialize_prob_crewmate(grid, num_crew)
      initialize_crewmate_pairs_probs(crewmate_pairs_probs)
      initialize_prob_alien_moved(grid)
      #print("INITIAL SHIP")
      #draw_ship_df(grid)
      #print("INITIAL CREW PROBS MATRIX")
      #draw_crewmate_probabilities_matrix(grid)
      #print("initial alien prob matrix")
      #draw_alien_probabilities_matrix(grid)
      #print("Sum of Alien Probabilities")
      #print(sum_probabilities_alien(grid))
      #print("Sum of Crewmate Probabilities")
      #print(sum_probabilities_crewmate(grid))
      #------------------------------
      centre_cells=[]
      if len(grid)%2==1:
        centre_cells.append([len(grid)//2,len(grid)//2])                             # Finding Center Cell
      else:
        centre_cells.append([len(grid)//2,len(grid)//2])
        centre_cells.append([len(grid)//2-1,len(grid)//2])
        centre_cells.append([len(grid)//2,len(grid)//2-1])
        centre_cells.append([len(grid)//2-1,len(grid)//2-1])
      if len(centre_cells)==1 and grid[centre_cells[0][0]][centre_cells[0][1]].open==False:     #Corner case closed center cell
        centre_cells_mod=grid[centre_cells[0][0]][centre_cells[0][1]].neighbors
        centre_cells=[ce.loc for ce in centre_cells_mod]
      #print("BOT CELL ", bot_cell.loc)
      #print("CENTER CELL ", centre_cells[0])
      if bot_cell.loc == centre_cells[0]:                                                 # Corner case where bot spawns on center cell
        #print("centre cells: ",centre_cells)
        #print("bot_cell: ",bot_cell.loc)
        #print("bot_cell spawned at center cell")
        max_prob_val=max(c.crewmate_probability for r in grid for c in r)             # Find max crew prob
        max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]   #Find cell of max prob
        #print("closest_max_prob_cell: ", max_prob_index)
        #draw_crewmate_probabilities_matrix(grid)
        path = find_path_bot2(bot_cell.loc, max_prob_index)                  # Find closest cell with max prob
        #print("bot_cell: ",bot_cell.loc)
        #print(path)
        msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path

      path = find_path_bot2(bot_cell.loc, centre_cells)                                 # Path to center cell
      #print("centre cells: ",centre_cells)
      #print("bot_cell: ",bot_cell.loc)
      #print(path)
      msg1, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)  #Moving on path

      #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)

      #print(msg1)
      msg2=""

      if (msg1=="The bot has rescued a crewmate"):
        crewmates.remove(bot_cell) # Removing the rescued crewmate cell from the crewmates array

        #print("NEW STARTING POSITION OF BOT IS",bot_cell.loc)

        bot_cell.crewmate_probability=0

        for row in grid:
          for cell in row:
            cell.visited=False
            cell.parent=None

        grid[bot_cell.x][bot_cell.y].value='B'
        bot_cell.value='B'

        #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
        max_prob_val=max(c.crewmate_probability for r in grid for c in r)
        max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
        max_prob_cell=random.choice(max_prob_index)
        # print(max_prob_cell)
        path=find_path_bot2(bot_cell.loc,[[max_prob_cell[0],max_prob_cell[1]]])
        #print("PATH TO SECOND CREW IS",path)
        #print("-----------------------------------------------------------------------------------------------------")
        #print(" Second crew member was found via path",path)

        for row in grid:
          for cell in row:
            cell.visited=False
            cell.parent=None

        #for cr in crewmates:
          #print(cr.loc)
        msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, alien_beep)
        #draw_ship_df(grid)
        #print(msg2)

      if (msg1=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
        alien_caught+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
        crew1_res+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
        crew_res_both+=1
        num_time_steps.append(time_steps)
      sim+=1
      #print("SIMULATION",sim,"TOOK",time_steps)
      #print("CURRENT NUM TIME STEPS ARRAY IS",num_time_steps)

      #print("END OF SIMULATION",sim,". RELOAD GRID.")

    #print("OUT OF",num_sims,"SIMULATIONS AND",num_sims*2,"CREWMATES")
    #print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")
    #print("AVERAGE NUMBER OF CREWMATES RESCUED PER SIMULATION",(crew1_res+crew_res_both*2)/num_sims)
    #if crew_res_both != 0:
      #print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
    #else:
      #print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESCUED")

      if crew_res_both != 0:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, sum(num_time_steps)/crew_res_both, crew_res_both, alien_caught])
      else:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, 0, crew_res_both, alien_caught])
      print(results)

      results_csv = pd.DataFrame(results)

In [ ]:
import pandas as pd
results = [[0.18483924814931874, 7, 1, 2, 2.0, 333.0, 1, 0], [0.18483924814931874, 7, 2, 4, 2.0, 254.5, 2, 0], [0.18483924814931874, 7, 3, 6, 2.0, 320.0, 3, 0], [0.18483924814931874, 7, 4, 8, 2.0, 318.75, 4, 0], [0.18483924814931874, 10, 1, 1, 1.0, 0, 0, 1], [0.18483924814931874, 10, 2, 3, 2.0, 416.0, 1, 1], [0.18483924814931874, 10, 3, 5, 2.333333333333333, 415.0, 2, 1]]
results_csv = pd.DataFrame(results)
results_csv.to_csv("results_bot_5.csv")

In [ ]:
bot_cell

BOT 6 Code

In [ ]:
def move_bot6(grid, bot_cell, path, aliens, crewmates, time_steps):
  #print("NORmAl")
  if bot_cell.value=='A' or bot_cell.value=='C*':
    return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
  while len(path) == 0:
    #print("NO PATH NOW SO ALIENS MOVE")
    status,crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
  # for next in path[1::]:
  while True:
    next=path[1]
    #print("MOVING THE BOT TO",next)
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1
    if (grid[next[0]][next[1]].value!='C' and grid[next[0]][next[1]].value!='C*' and grid[next[0]][next[1]].value!='A'):
      grid[next[0]][next[1]].value='B'
      next_cell.value='B'
      next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    if bot_cell.value=='A' or bot_cell.value=='C*':
      #print("ALIENS CAUGHT BOT AT",bot_cell.loc)
      #draw_ship_df(grid)
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    if bot_cell.value=='C':
      # grid[next[0]][next[1]].value='B'
      # next_cell.value='B'
      #print("BOT FOUND CREWMATE AT",bot_cell.loc)
      #draw_ship_df(grid)
      bot_cell.value='B'
      #draw_ship_df(grid)
      return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)

    bot_cell.crewmate_probability=0

    #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    # print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    #print("BOT MOVED")
    #draw_ship_df(grid)

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("AFTER TAKE STEP")

    #print("ALIEN MOVED")
    #draw_ship_df(grid)

    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    if bot_cell.value=='A' or bot_cell.value=='C*':
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    #print("PATH GOING TO BE FOLLOWED BY BOT3 IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

In [ ]:
# BOT 6 DRIVER CODE

bot_id = 6
size = 30
num_aliens = 2
num_crew = 2
detector_size_values = [5, 7, 10]
alpha_10_percent = -2*math.log(.1)/(size/2)
alpha_25_percent = -2*math.log(.25)/(size/2)
alpha_50_percent = -2*math.log(.5)/(size/2)
alpha_values = [alpha_10_percent, alpha_25_percent, alpha_50_percent]                                                               # Caluclating alpha based on ship size
print("Alpha: ", alpha_50_percent)
num_sim = 30
results = []
for alpha in alpha_values:
  for detector_size in detector_size_values:
    sim=0
    alien_caught=0
    crew1_res=0
    crew_res_both=0
    num_time_steps=[]
    for _ in range(num_sim):
      print(sim)
      alien_beep = False
      time_steps=1
      #crewmate_pairs_probs = {}
      crewmates = []
      grid = create_grid(size, size)
      for row in grid:
        for cell in row:
          cell.neighbors = get_neighbors(cell, grid)
      # bot_cell = create_bot(grid)
      aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

      # crew_start_cell = random.choice(detect_open_cells(grid))
      # if (crew_start_cell.x==bot_cell.x and crew_start_cell.y==bot_cell.y):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # if (crew_start_cell.x==crew_cell.x and crew_start_cell.y==crew_cell.y):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # while(crew_start_cell==bot_cell or crew_start_cell==crew_cell):
      #   crew_start_cell = random.choice(detect_open_cells(grid))
      # crew_start_cell.value = 'C'

      grid=initialize_cell_neighbors(grid)
      num_open_cells = len(detect_open_cells(grid))
      for row in grid:
        for cell in row:
          cell.open = detect_if_cell_open(cell)
          cell.neighbors = get_open_neighbors(cell, grid)
      # initialize_prob_alien(grid, num_aliens)

      distance_map={}
      pred_map={}
      for i in range(len(grid)):
        for j in range(len(grid[0])):
          s=(i,j)
          dlist,f=dijkstra(grid,s)
          distance_map[s]=dlist
          pred_map[s]=f

      # initialize_crewmate_pairs_probs(crewmate_pairs_probs)
      # print(crewmate_pairs_probs)
      initialize_prob_crewmate(grid, num_crew)
      initialize_prob_alien_moved(grid)
      # bot_cell = create_bot(grid)
      # draw_ship_df(grid)
      # print("INITIAL CREW PROBS MATRIX")
      # draw_crewmate_probabilities_matrix(grid)
      # print("initial alien prob matrix")
      # draw_alien_probabilities_matrix(grid)
      # print("Sum of Alien Probabilities")
      # print(sum_probabilities_alien(grid))
      # print("Sum of Crewmate Probabilities")
      # print(sum_probabilities_crewmate(grid))

      # for r in grid:
      #   for c in r:
      #     for r2 in grid:
      #       for c2 in r2:
      #         print("PAIRWISE PROB FOR",c.loc,"AND",c2.loc,"IS",get_value_crewmate_pairs(c.x,c.y,c2.x,c2.y))

      # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
      max_prob_val=max(c.crewmate_probability for r in grid for c in r)
      max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
      max_prob_cell=random.choice(max_prob_index)
      path=find_path(bot_cell.loc,max_prob_cell)

      for row in grid:
        for cell in row:
          cell.visited=False
          cell.parent=None

      # print("-----------------------------------------------------------------------------------------------------")
      # print("crew member was found via path",path)
      # if len(path)!=0:
      #   print("Now moving Bot to rescue Crewmate...")

      msg1, bot_cell, crewmates, grid, aliens,time_steps = move_bot6(grid, bot_cell, path, aliens, crewmates, time_steps)
      # print(msg1)

      if (msg1=="The bot has rescued a crewmate"):
        # print("NEW STARTING POSITION OF BOT IS",bot_cell)
        # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
        max_prob_val=max(c.crewmate_probability for r in grid for c in r)
        max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
        max_prob_cell=random.choice(max_prob_index)
        path=find_path(bot_cell.loc,max_prob_cell)

        # print("-----------------------------------------------------------------------------------------------------")
        # print(" Second crew member was found via path",path)

        for row in grid:
          for cell in row:
            cell.visited=False
            cell.parent=None

        msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot6(grid, bot_cell, path, aliens, crewmates, time_steps)
        # print(msg2)

      if (msg1=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
        alien_caught+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
        crew1_res+=1
      if (msg1=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
        crew_res_both+=1
        num_time_steps.append(time_steps)
      sim+=1
      # print("END OF SIMULATION. RELOAD GRID.")

    # print("OUT OF 30 SIMULATIONS AND 60 CREWMATES")
    # print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")
    # if crew_res_both != 0:
    #   print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
    # else:
    #   print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESQUED")
      if crew_res_both != 0:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, sum(num_time_steps)/crew_res_both, crew_res_both, alien_caught])
      else:
        results.append([alpha, detector_size, sim, crew1_res+crew_res_both*2, crew1_res+crew_res_both*2/sim, 0, crew_res_both, alien_caught])
      print(results)

      results_csv = pd.DataFrame(results)
      results_csv.to_csv("results_bot_6.csv")


In [ ]:
import pandas as pd
results = [[0.3070113457325394, 5, 1, 2, 2.0, 298.0, 1, 0], [0.3070113457325394, 5, 2, 3, 2.0, 298.0, 1, 1], [0.3070113457325394, 5, 3, 4, 2.6666666666666665, 298.0, 1, 2], [0.3070113457325394, 5, 4, 5, 3.5, 298.0, 1, 3], [0.3070113457325394, 5, 5, 5, 3.4, 298.0, 1, 4]]
results_csv = pd.DataFrame(results)
results_csv.to_csv("results_bot_6.csv")

In [ ]:
def move_bot7(grid, bot_cell, path, aliens, crewmates, time_steps):
  #print("NORmAl")
  #if bot_cell.value=='A' or bot_cell.value=='C*':
  for alien_cell in aliens:
    if bot_cell == alien_cell:
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
  while len(path) == 0:
    print("NO PATH NOW SO ALIENS MOVE")
    status,crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
  # for next in path[1::]:
  while True:
    next=path[1]
    print("MOVING THE BOT TO",next)
    #print("START OF THE FOR LOOP")
    next_cell = grid[next[0]][next[1]]
    bot_cell.value=1
    grid[bot_cell.x][bot_cell.y].value=1
    #if (grid[next[0]][next[1]].value!='C' and grid[next[0]][next[1]].value!='C*' and grid[next[0]][next[1]].value!='A'):
    for alien_cell in aliens:
      if (grid[next[0]][next[1]] != alien_cell):
        for crew_cell in crewmates:
          if (grid[next[0]][next[1]] != crew_cell):
            grid[next[0]][next[1]].value='B'
            next_cell.value='B'
            next_cell.open = True

    bot_cell=next_cell
    #print("STOP STOP sTOP")
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        print("ALIENS CAUGHT BOT AT",bot_cell.loc)
        draw_ship_df(grid)
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for crew_cell in crewmates:
      if bot_cell == crew_cell:
        # grid[next[0]][next[1]].value='B'
        # next_cell.value='B'
        bot_cell.crewmate_probability = 0
        print("BOT FOUND CREWMATE AT",bot_cell.loc)
        draw_ship_df(grid)
        bot_cell.value='B'
        #draw_ship_df(grid)
        return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

    #draw_ship_df(grid)
    # print("NEW BOT LOC 1",grid[next[0]][next[1]].loc)
    # print("NEW BOT LOC 2",bot_cell.loc)

    bot_cell.crewmate_probability=0

    #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)
    # print("PATH  RETURNED BY BFS IS",path)

    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    grid[next[0]][next[1]].value='B'
    next_cell.value='B'

    print("BOT MOVED")
    draw_ship_df(grid)

    #print("BEFORE TAKE STEP")
    status, crewmates, aliens,time_steps, alien_beep = take_step(grid, aliens, crewmates, bot_cell,time_steps)
    #print("AFTER TAKE STEP")

    #draw_crewmate_probabilities_matrix(grid)
    print("ALIEN MOVED")
    draw_ship_df(grid)

    if status == "The bot was caught by the aliens!":
      return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    for alien_cell in aliens:
      if bot_cell == alien_cell:
        return "The bot was caught by the aliens!", bot_cell, crewmates, grid, aliens,time_steps
    print("PATH GOING TO BE FOLLOWED BY BOT7 IS",path)
    #print("CHECK CHECK CHECK")
  #print("REACHED OUTSIDE THE FOR LOOP")
  return "The bot has rescued a crewmate", bot_cell, crewmates, grid, aliens,time_steps

In [ ]:
# BOT 7 DRIVER CODE
bot_id = 7
size = 7
num_aliens = 2
num_crew = 2
detector_size = 1
alpha = -2*math.log(.1)/size                                                                  # Caluclating alpha based on ship size
print("Alpha: ", alpha)
crewmate_pairs_probs = {}
crewmates = []
num_time_steps=[]
sim=0
crew1_res=0
crew_res_both=0
alien_caught=0
for _ in range(100):
  alien_beep = False
  crewmates = []
  crewmate_pairs_probs = []
  alien_pairs_probs = {}
  #celli_given_cellj_and_cellk = {}
  crewmate_pairs_probs = {}
  grid = create_grid(size, size)
  for row in grid:
    for cell in row:
      cell.neighbors = get_neighbors(cell, grid)
  # bot_cell = create_bot(grid)
  aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)

  grid=initialize_cell_neighbors(grid)
  num_open_cells = len(detect_open_cells(grid))
  for row in grid:
    for cell in row:
      cell.open = detect_if_cell_open(cell)
      cell.neighbors = get_open_neighbors(cell, grid)
  distance_map={}
  pred_map={}
  for i in range(len(grid)):
    for j in range(len(grid[0])):
      s=(i,j)
      dlist,f=dijkstra(grid,s)
      distance_map[s]=dlist
      pred_map[s]=f

  print(distance_map)
  # initialize_prob_alien(grid, num_aliens)
  initialize_prob_crewmate(grid, num_crew)
  initialize_crewmate_pairs_probs(crewmate_pairs_probs)
  initialize_alien_pairs_probs(alien_pairs_probs, grid)
  initialize_prob_alien_moved(grid)
  # bot_cell = create_bot(grid)
  draw_ship_df(grid)
  print("INITIAL CREW PROBS MATRIX")
  draw_crewmate_probabilities_matrix(grid)
  print("initial alien prob matrix")
  draw_alien_probabilities_matrix(grid)
  print("Sum of Alien Probabilities")
  print(sum_probabilities_alien(grid))
  print("Sum of Crewmate Probabilities")
  print(sum_probabilities_crewmate(grid))


  # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
  max_prob_val=max(c.crewmate_probability for r in grid for c in r)
  max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
  max_prob_cell=random.choice(max_prob_index)
  path=find_path(bot_cell.loc,max_prob_cell)

  for row in grid:
    for cell in row:
      cell.visited=False
      cell.parent=None


  if len(path)!=0:
    print("Now moving Bot to rescue Crewmate...")

  msg1, bot_cell, crewmates, grid, aliens,time_steps = move_bot7(grid, bot_cell, path, aliens, crewmates, time_steps)
  print(msg1)

  if (msg1=="The bot has rescued a crewmate"):
    print("NEW STARTING POSITION OF BOT IS",bot_cell)
    # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)


    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot7(grid, bot_cell, path, aliens, crewmates, time_steps)
    print(msg2)
    print("Final crewmate probs matrix:")
    draw_crewmate_probabilities_matrix(grid)

  if (msg1=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
    alien_caught+=1
  if (msg1=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
    crew1_res+=1
  if (msg1=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
    crew_res_both+=1
    num_time_steps.append(time_steps)

  print("END OF SIMULATION. RELOAD GRID.")

print("OUT OF 30 SIMULATIONS AND 60 CREWMATES")
print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")
if crew_res_both != 0:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
else:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESCUED")

In [ ]:
draw_ship_df(grid)

In [ ]:
draw_crewmate_probabilities_matrix(grid)

Bot 8 Code


In [ ]:
bot_id=8
size=5
num_aliens = 2
num_crew = 2
detector_size = 1
alpha = -2*math.log(.1)/size                                                                  # Caluclating alpha based on ship size
print("Alpha: ", alpha)
num_time_steps=[]
sim=0
crew1_res=0
crew_res_both=0
alien_caught=0
for _ in range(100):
  #alien_beep = False
  beep1=False
  crewmates = []
  crewmate_pairs_probs = {}
  alien_pairs_probs = {}
  time_steps=1
  grid = create_grid(size, size)
  for row in grid:
    for cell in row:
      cell.neighbors = get_neighbors(cell, grid)
  aliens, crewmates, bot_cell = explore_grid(size, grid, num_aliens, num_crew, detector_size)
  print("Crewmate 1 location: ")
  print(crewmates[0].loc)
  print("Crewmate 2 location: ")
  print(crewmates[1].loc)
  print("Aliens location: ")
  print(aliens[0].loc)
  grid=initialize_cell_neighbors(grid)
  num_open_cells = len(detect_open_cells(grid))
  for row in grid:
    for cell in row:
      cell.open = detect_if_cell_open(cell)
      cell.neighbors = get_open_neighbors(cell, grid)
  distance_map={}
  pred_map={}
  for i in range(len(grid)):
    for j in range(len(grid[0])):
      s=(i,j)
      dlist,f=dijkstra(grid,s)
      distance_map[s]=dlist
      pred_map[s]=f
  initialize_prob_crewmate(grid, num_crew)
  initialize_prob_alien_moved(grid)
  print("INITIAL SHIP")
  draw_ship_df(grid)
  print("INITIAL CREW PROBS MATRIX")
  draw_crewmate_probabilities_matrix(grid)
  print("initial alien prob matrix")
  draw_alien_probabilities_matrix(grid)
  print("Sum of Alien Probabilities")
  print(sum_probabilities_alien(grid))
  print("Sum of Crewmate Probabilities")
  print(sum_probabilities_crewmate(grid))
  #------------------------------
  centre_cells=[]
  if len(grid)%2==1:
    centre_cells.append([len(grid)//2,len(grid)//2])                             # Finding Center Cell
  else:
    centre_cells.append([len(grid)//2,len(grid)//2])
    centre_cells.append([len(grid)//2-1,len(grid)//2])
    centre_cells.append([len(grid)//2,len(grid)//2-1])
    centre_cells.append([len(grid)//2-1,len(grid)//2-1])
  if len(centre_cells)==1 and grid[centre_cells[0][0]][centre_cells[0][1]].open==False:     #Corner case closed center cell
    centre_cells_mod=grid[centre_cells[0][0]][centre_cells[0][1]].neighbors
    centre_cells=[ce.loc for ce in centre_cells_mod]
  print("BOT CELL ", bot_cell.loc)
  print("CENTER CELL ", centre_cells[0])
  if bot_cell.loc == centre_cells[0]:                                                 # Corner case where bot spawns on center cell
    print("centre cells: ",centre_cells)
    print("bot_cell: ",bot_cell.loc)
    print("bot_cell spawned at center cell")
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)             # Find max crew prob
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]   #Find cell of max prob
    print("closest_max_prob_cell: ", max_prob_index)
    draw_crewmate_probabilities_matrix(grid)
    path = find_path_bot2(bot_cell.loc, max_prob_index)                  # Find closest cell with max prob
    print("bot_cell: ",bot_cell.loc)
    print(path)
    msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, beep1)  #Moving on path

  path = find_path_bot2(bot_cell.loc, centre_cells)                                 # Path to center cell
  print("centre cells: ",centre_cells)
  print("bot_cell: ",bot_cell.loc)
  print(path)


  msg, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, beep1)  #Moving on path
  if msg=="reached centre":
    print("REACHED THE CENTRE")
  elif msg=="entered a high prob region":
    print("REACHED AN AREA WITH PROB >0.25")
  #res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)

  print(msg)
  sim+=1
  print("Simmulation Number: ", sim)
  if (msg=="The bot was caught by the aliens!"):
    alien_caught+=1
  elif (msg=="The bot has rescued a crewmate"):
    print("NEW STARTING POSITION OF BOT IS",bot_cell)
    # res, path, time_steps = breadth_first_search_bot_1(grid,bot_cell, bot_cell, aliens, time_steps, crewmates)
    max_prob_val=max(c.crewmate_probability for r in grid for c in r)
    max_prob_index=[(i,j) for i,row in enumerate(grid) for j,val in enumerate(row) if val.crewmate_probability==max_prob_val]
    max_prob_cell=random.choice(max_prob_index)
    path=find_path(bot_cell.loc,max_prob_cell)


    for row in grid:
      for cell in row:
        cell.visited=False
        cell.parent=None

    msg2, bot_cell, crewmates, grid, aliens,time_steps = move_bot2(grid, bot_cell, path, aliens, crewmates, time_steps, beep1)
    print(msg2)
    print("Final crewmate probs matrix:")
    draw_crewmate_probabilities_matrix(grid)
  if (msg=="The bot was caught by the aliens!" or msg2=="The bot was caught by the aliens!"):
    alien_caught+=1
  if (msg=="The bot has rescued a crewmate" and msg2=="The bot was caught by the aliens!"):
    crew1_res+=1
  if (msg=="The bot has rescued a crewmate" and msg2=="The bot has rescued a crewmate"):
    crew_res_both+=1
    num_time_steps.append(time_steps)

  print("END OF SIMULATION. RELOAD GRID.")

print("OUT OF 30 SIMULATIONS AND 60 CREWMATES")
print("THE BOT RESCUED",crew1_res+crew_res_both*2,"CREWMATES")
if crew_res_both != 0:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS",sum(num_time_steps)/crew_res_both)
else:
  print("AVERAGE NUMBER OF MOVES NEEDED TO RESCUE ALL CREWMATES IS NONE RESCUED")


In [ ]:
path

In [ ]:
curr_v